In [1]:
!pip install kaggle

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import pickle
import os
from datetime import datetime
import pyarrow.parquet as pq
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.cuda.amp import autocast, GradScaler


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("titericz/leap-dataset-giba-part-2")

print("Path to dataset files:", path)

100%|██████████| 15.7G/15.7G [12:23<00:00, 22.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/titericz/leap-dataset-giba-part-2/versions/2


In [25]:
# copy one train file to cotent
!cp /root/.cache/kagglehub/datasets/titericz/leap-dataset-giba-part-2/versions/2/train_batch/10.parquet /content/

In [37]:
import kagglehub
import os

# Define the model handle
model_handle = "tianffan/transformer_signal_end_to_end_leap/transformers/default"

# Download the model
path = kagglehub.model_download(model_handle)

print(f"Path to downloaded model files: {path}")

# Define the destination directory
destination_dir = '/content/'

# Copy all files from the downloaded model directory to the destination directory
# Use shell command as it's generally more efficient for copying directories
!cp -r "$path"/* "$destination_dir"

print(f"Successfully copied files from '{path}' to '{destination_dir}'")

Path to downloaded model files: /kaggle/input/transformer_signal_end_to_end_leap/transformers/default/2
Successfully copied files from '/kaggle/input/transformer_signal_end_to_end_leap/transformers/default/2' to '/content/1'


In [27]:
"""
LEAP数据加载与探索
加载完整数据并按90/10划分训练集和测试集
"""

import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 配置：数据路径
# ============================================================================

# 数据集路径
DATASET_PATHS = [
    '/content',
]

TRAIN_SPLIT = 0.5  # 前90%用于训练

# ============================================================================
# 加载数据 - 完整加载并按顺序划分
# ============================================================================

print("=" * 80)
print("🔍 LEAP数据加载 - 90/10顺序划分")
print("=" * 80)

all_data = []
total_samples = 0

for dataset_path in DATASET_PATHS:
    path = Path(dataset_path)
    if not path.exists():
        print(f"⚠️  路径不存在: {dataset_path}")
        continue

    # 获取所有parquet文件
    parquet_files = sorted(path.glob('*.parquet'))
    print(f"\n📂 {path.name}: 找到 {len(parquet_files)} 个文件")

    for file in tqdm(parquet_files, desc=f"  加载{path.name}"):
        # 读取完整文件
        df_full = pd.read_parquet(file)
        total_samples += len(df_full)

        all_data.append(df_full)

        print(f"    {file.name}: {len(df_full):,} 样本")

# 合并所有数据（保持顺序）
full_df = pd.concat(all_data, ignore_index=True)

print(f"\n✅ 数据加载完成!")
print(f"  总样本数: {total_samples:,}")
print(f"  合并后数据形状: {full_df.shape}")

# ============================================================================
# 按顺序划分训练集和测试集（90/10）
# ============================================================================

print("\n" + "=" * 80)
print("✂️  数据划分 (90% Train / 10% Test)")
print("=" * 80)

# 按顺序划分
split_idx = int(len(full_df) * TRAIN_SPLIT)

train_df = full_df.iloc[:split_idx].copy()
test_df = full_df.iloc[split_idx:].copy()

print(f"\n【划分结果】")
print(f"  训练集 (前90%): {len(train_df):,} 样本")
print(f"  测试集 (后10%): {len(test_df):,} 样本")
print(f"  划分比例: {len(train_df)/len(full_df)*100:.1f}% / {len(test_df)/len(full_df)*100:.1f}%")

# 检查index连续性
print(f"\n【Index连续性检查】")
if hasattr(train_df.index, 'is_monotonic_increasing'):
    train_continuous = train_df.index.is_monotonic_increasing
    test_continuous = test_df.index.is_monotonic_increasing
    print(f"  训练集index连续: {'✅' if train_continuous else '❌'}")
    print(f"  测试集index连续: {'✅' if test_continuous else '❌'}")

print(f"  训练集index范围: [{train_df.index.min()}, {train_df.index.max()}]")
print(f"  测试集index范围: [{test_df.index.min()}, {test_df.index.max()}]")

# 释放内存
del full_df
import gc
gc.collect()

# ============================================================================
# 数据探索 - 查看训练集情况
# ============================================================================

print("\n" + "=" * 80)
print("📊 训练集数据基本信息")
print("=" * 80)

# 1. 基本信息
print(f"\n【数据维度】")
print(f"  训练集样本数: {len(train_df):,}")
print(f"  测试集样本数: {len(test_df):,}")
print(f"  特征数: {len(train_df.columns)}")
print(f"  训练集内存: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  测试集内存: {test_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 2. 列名分类
print(f"\n【特征分类】")
all_cols = train_df.columns.tolist()

# 输入特征
input_patterns = ['state_', 'pbuf_', 'cam_in_']
input_cols = [col for col in all_cols if any(col.startswith(p) for p in input_patterns)]

# 输出特征
output_patterns = ['ptend_', 'cam_out_']
output_cols = [col for col in all_cols if any(col.startswith(p) for p in output_patterns)]

print(f"  输入特征 (INPUT):  {len(input_cols)} 个")
for pattern in input_patterns:
    count = len([c for c in input_cols if c.startswith(pattern)])
    if count > 0:
        print(f"    - {pattern}*: {count} 个")

print(f"\n  输出特征 (OUTPUT): {len(output_cols)} 个")
for pattern in output_patterns:
    count = len([c for c in output_cols if c.startswith(pattern)])
    if count > 0:
        print(f"    - {pattern}*: {count} 个")

other_cols = [c for c in all_cols if c not in input_cols and c not in output_cols]
if other_cols:
    print(f"\n  其他列: {len(other_cols)} 个")
    print(f"    {other_cols[:5]}")

# 3. 数据类型
print(f"\n【数据类型】")
print(train_df.dtypes.value_counts().to_dict())

# 4. 缺失值检查
print(f"\n【缺失值】")
missing_train = train_df.isnull().sum()
missing_test = test_df.isnull().sum()

if missing_train.sum() == 0:
    print("  训练集: ✅ 无缺失值")
else:
    print(f"  训练集: ⚠️  发现缺失值:")
    print(missing_train[missing_train > 0].head(10))

if missing_test.sum() == 0:
    print("  测试集: ✅ 无缺失值")
else:
    print(f"  测试集: ⚠️  发现缺失值:")
    print(missing_test[missing_test > 0].head(10))

# 5. 输入特征统计（训练集）
print(f"\n【输入特征统计 - 训练集】")
input_stats = train_df[input_cols].describe()
print(f"  均值范围: [{input_stats.loc['mean'].min():.2e}, {input_stats.loc['mean'].max():.2e}]")
print(f"  标准差范围: [{input_stats.loc['std'].min():.2e}, {input_stats.loc['std'].max():.2e}]")
print(f"  最小值范围: [{input_stats.loc['min'].min():.2e}, {input_stats.loc['min'].max():.2e}]")
print(f"  最大值范围: [{input_stats.loc['max'].min():.2e}, {input_stats.loc['max'].max():.2e}]")

# 6. 输出特征统计（训练集）
print(f"\n【输出特征统计 - 训练集】")
output_stats = train_df[output_cols].describe()
print(f"  均值范围: [{output_stats.loc['mean'].min():.2e}, {output_stats.loc['mean'].max():.2e}]")
print(f"  标准差范围: [{output_stats.loc['std'].min():.2e}, {output_stats.loc['std'].max():.2e}]")
print(f"  最小值范围: [{output_stats.loc['min'].min():.2e}, {output_stats.loc['min'].max():.2e}]")
print(f"  最大值范围: [{output_stats.loc['max'].min():.2e}, {output_stats.loc['max'].max():.2e}]")

# 7. 示例数据
print(f"\n【训练集数据预览】前5个输入特征 + 前5个输出特征")
preview_cols = input_cols[:5] + output_cols[:5]
print(train_df[preview_cols].head())

print(f"\n【测试集数据预览】")
print(test_df[preview_cols].head())

# 8. 详细列名列表
print(f"\n【完整列名列表】")
print(f"\n输入特征 ({len(input_cols)}个):")
for i, col in enumerate(input_cols, 1):
    print(f"  {i:3d}. {col}")

print(f"\n输出特征 ({len(output_cols)}个):")
for i, col in enumerate(output_cols, 1):
    print(f"  {i:3d}. {col}")

# ============================================================================
# 时间连续性验证（重要！）
# ============================================================================

print("\n" + "=" * 80)
print("⏰ 时间连续性验证")
print("=" * 80)

# 检查相邻样本的变化幅度（判断是否时间连续）
print("\n【相邻样本变化检查】")
sample_feature = input_cols[0]  # 用第一个特征检查

train_diff = train_df[sample_feature].diff().abs()
test_diff = test_df[sample_feature].diff().abs()

print(f"  特征: {sample_feature}")
print(f"  训练集相邻样本平均变化: {train_diff.mean():.6e}")
print(f"  训练集相邻样本最大变化: {train_diff.max():.6e}")
print(f"  测试集相邻样本平均变化: {test_diff.mean():.6e}")
print(f"  测试集相邻样本最大变化: {test_diff.max():.6e}")

# 检查train和test交界处的跳跃
boundary_jump = abs(test_df[sample_feature].iloc[0] - train_df[sample_feature].iloc[-1])
print(f"\n  训练/测试边界跳跃: {boundary_jump:.6e}")

if boundary_jump > train_diff.quantile(0.99):
    print(f"  ⚠️  边界跳跃较大，可能数据不完全连续")
else:
    print(f"  ✅ 边界跳跃正常，数据基本连续")

print("\n" + "=" * 80)
print("✅ 数据探索完成！train_df 和 test_df 已准备好")
print("=" * 80)

# ============================================================================
# 保存变量供后续使用
# ============================================================================

# 将关键变量导出
INPUT_FEATURES = input_cols
OUTPUT_FEATURES = output_cols

print(f"\n💾 已创建全局变量:")
print(f"  - train_df: 训练数据 {train_df.shape}")
print(f"  - test_df: 测试数据 {test_df.shape}")
print(f"  - INPUT_FEATURES: 输入特征列表 (长度: {len(INPUT_FEATURES)})")
print(f"  - OUTPUT_FEATURES: 输出特征列表 (长度: {len(OUTPUT_FEATURES)})")

print("\n" + "=" * 80)
print("🎯 数据准备流程:")
print("  1. ✅ 数据加载")
print("  2. ✅ 90/10划分（按顺序）")
print("  3. ✅ 特征识别")
print("  4. ✅ 连续性验证")
print("  5. ⏭️  可以开始训练SST模型")
print("=" * 80)

🔍 LEAP数据加载 - 90/10顺序划分

📂 content: 找到 2 个文件


  加载content:   0%|          | 0/2 [00:00<?, ?it/s]

    10.parquet: 625,000 样本
    11.parquet: 625,000 样本

✅ 数据加载完成!
  总样本数: 1,250,000
  合并后数据形状: (1250000, 793)

✂️  数据划分 (90% Train / 10% Test)

【划分结果】
  训练集 (前90%): 625,000 样本
  测试集 (后10%): 625,000 样本
  划分比例: 50.0% / 50.0%

【Index连续性检查】
  训练集index连续: ✅
  测试集index连续: ✅
  训练集index范围: [0, 624999]
  测试集index范围: [625000, 1249999]

📊 训练集数据基本信息

【数据维度】
  训练集样本数: 625,000
  测试集样本数: 625,000
  特征数: 793
  训练集内存: 1988.41 MB
  测试集内存: 1988.41 MB

【特征分类】
  输入特征 (INPUT):  490 个
    - state_*: 361 个
    - pbuf_*: 120 个
    - cam_in_*: 9 个

  输出特征 (OUTPUT): 302 个
    - ptend_*: 294 个
    - cam_out_*: 8 个

  其他列: 1 个
    ['sample_id']

【数据类型】
{dtype('float32'): 751, dtype('float64'): 41, dtype('int32'): 1}

【缺失值】
  训练集: ✅ 无缺失值
  测试集: ✅ 无缺失值

【输入特征统计 - 训练集】
  均值范围: [-2.68e+03, 4.12e+02]
  标准差范围: [0.00e+00, 4.58e+03]
  最小值范围: [-3.17e+04, 2.26e+02]
  最大值范围: [2.80e-48, 3.54e+03]

【输出特征统计 - 训练集】
  均值范围: [-8.37e-01, 4.99e+00]
  标准差范围: [0.00e+00, 3.92e+00]
  最小值范围: [-3.05e+03, 9.01e-01]
  最大值范围: [0.00e+00, 7.72e+

In [28]:

# 方式1: 手动列出要删除的特征名称
FEATURES_TO_REMOVE = [
    # ❌ 100%零值的输出特征（必删！）
    'ptend_q0003_0',
    'ptend_u_0',
    'ptend_q0002_0',
    'ptend_v_0',
    'ptend_q0001_0',

    # 🚨 新增：删除所有ptend_q相关信号（基于您的诊断报告）
    # ptend_q0001 系列
    'ptend_q0001_12', 'ptend_q0001_13', 'ptend_q0001_14', 'ptend_q0001_15', 'ptend_q0001_16',
    'ptend_q0001_17', 'ptend_q0001_18', 'ptend_q0001_19', 'ptend_q0001_20', 'ptend_q0001_21',
    'ptend_q0001_22', 'ptend_q0001_23', 'ptend_q0001_24', 'ptend_q0001_25', 'ptend_q0001_26',
    'ptend_q0001_27', 'ptend_q0001_28', 'ptend_q0001_29', 'ptend_q0001_30', 'ptend_q0001_31',
    'ptend_q0001_32', 'ptend_q0001_33', 'ptend_q0001_34', 'ptend_q0001_35', 'ptend_q0001_36',
    'ptend_q0001_37', 'ptend_q0001_38', 'ptend_q0001_39', 'ptend_q0001_40', 'ptend_q0001_41',
    'ptend_q0001_42', 'ptend_q0001_43', 'ptend_q0001_44', 'ptend_q0001_45', 'ptend_q0001_46',
    'ptend_q0001_47', 'ptend_q0001_48', 'ptend_q0001_49', 'ptend_q0001_50', 'ptend_q0001_51',
    'ptend_q0001_52', 'ptend_q0001_53', 'ptend_q0001_54', 'ptend_q0001_55', 'ptend_q0001_56',
    'ptend_q0001_57', 'ptend_q0001_58', 'ptend_q0001_59',

    # ptend_q0002 系列
    'ptend_q0002_23', 'ptend_q0002_24', 'ptend_q0002_25', 'ptend_q0002_26', 'ptend_q0002_27',
    'ptend_q0002_28', 'ptend_q0002_29', 'ptend_q0002_30', 'ptend_q0002_31', 'ptend_q0002_32',
    'ptend_q0002_33', 'ptend_q0002_34', 'ptend_q0002_35', 'ptend_q0002_36', 'ptend_q0002_37',
    'ptend_q0002_38', 'ptend_q0002_39', 'ptend_q0002_40', 'ptend_q0002_41', 'ptend_q0002_42',
    'ptend_q0002_43', 'ptend_q0002_44', 'ptend_q0002_45', 'ptend_q0002_46', 'ptend_q0002_47',
    'ptend_q0002_48', 'ptend_q0002_49', 'ptend_q0002_50', 'ptend_q0002_51', 'ptend_q0002_52',
    'ptend_q0002_53', 'ptend_q0002_54', 'ptend_q0002_55', 'ptend_q0002_56', 'ptend_q0002_57',
    'ptend_q0002_58', 'ptend_q0002_59',

    # ptend_q0003 系列
    'ptend_q0003_12', 'ptend_q0003_13', 'ptend_q0003_14', 'ptend_q0003_15', 'ptend_q0003_16',
    'ptend_q0003_17', 'ptend_q0003_18', 'ptend_q0003_19', 'ptend_q0003_20', 'ptend_q0003_21',
    'ptend_q0003_22', 'ptend_q0003_23', 'ptend_q0003_24', 'ptend_q0003_25', 'ptend_q0003_26',
    'ptend_q0003_27', 'ptend_q0003_28', 'ptend_q0003_29', 'ptend_q0003_30', 'ptend_q0003_31',
    'ptend_q0003_32', 'ptend_q0003_33', 'ptend_q0003_34', 'ptend_q0003_35', 'ptend_q0003_36',
    'ptend_q0003_37', 'ptend_q0003_38', 'ptend_q0003_39', 'ptend_q0003_40', 'ptend_q0003_41',
    'ptend_q0003_42', 'ptend_q0003_43', 'ptend_q0003_44', 'ptend_q0003_45', 'ptend_q0003_46',
    'ptend_q0003_47', 'ptend_q0003_48', 'ptend_q0003_49', 'ptend_q0003_50', 'ptend_q0003_51',
    'ptend_q0003_52', 'ptend_q0003_53', 'ptend_q0003_54', 'ptend_q0003_55', 'ptend_q0003_56',
    'ptend_q0003_57', 'ptend_q0003_58', 'ptend_q0003_59',

    # ⚠️ 可选：添加其他你想删除的特征
    # 'state_q0002_16',  # 例如：高零值的输入特征
    # 'state_q0002_15',

    # 💡 提示：
    # 1. 查看上面Cell 1生成的CSV文件
    # 2. 复制要删除的column_name到这里
    # 3. 一行一个，用逗号分隔
]

# 方式2: 自动根据阈值删除（可选，取消注释使用）
# AUTO_REMOVE_CONFIG = {
#     'output_zero_threshold': 0.90,  # 输出特征：零值≥90%删除
#     'input_zero_threshold': 0.95,   # 输入特征：零值≥95%删除
#     'all_zero_threshold': 1.0,      # 所有特征：零值=100%删除
# }

# ============================================================================
# 应用删除操作
# ============================================================================

def apply_feature_removal(df, features_to_remove, input_features, output_features, save_dir='/content/'):
    """
    应用特征删除操作

    Args:
        df: 原始DataFrame (df_train)
        features_to_remove: 要删除的特征列表
        input_features: 原始输入特征列表
        output_features: 原始输出特征列表
        save_dir: 保存目录

    Returns:
        df_clean: 清洗后的DataFrame
        input_features_clean: 清洗后的输入特征列表
        output_features_clean: 清洗后的输出特征列表
        removal_report: 删除报告
    """
    print("="*80)
    print("📝 Cell 2: 应用特征删除")
    print("="*80)

    # 验证特征是否存在
    features_to_remove_valid = []
    features_not_found = []

    for feature in features_to_remove:
        if feature in df.columns:
            features_to_remove_valid.append(feature)
        else:
            features_not_found.append(feature)

    if features_not_found:
        print(f"\n⚠️  警告：以下特征不在DataFrame中，将被忽略:")
        for f in features_not_found:
            print(f"   - {f}")

    # 区分输入和输出
    input_to_remove = [f for f in features_to_remove_valid if f in input_features]
    output_to_remove = [f for f in features_to_remove_valid if f in output_features]

    print(f"\n📊 删除统计:")
    print(f"   总删除: {len(features_to_remove_valid)} 个特征")
    print(f"   - 输入特征: {len(input_to_remove)} 个")
    print(f"   - 输出特征: {len(output_to_remove)} 个")

    # # 显示要删除的特征
    # if output_to_remove:
    #     print(f"\n❌ 删除的输出特征 ({len(output_to_remove)}个):")
    #     for i, f in enumerate(output_to_remove, 1):
    #         if f in zero_stats_df['column_name'].values:
    #             zero_pct = zero_stats_df[zero_stats_df['column_name']==f]['zero_percentage'].values[0]
    #             print(f"   {i:2d}. {f:40s} (零值: {zero_pct:.1f}%)")
    #         else:
    #             print(f"   {i:2d}. {f}")

    # if input_to_remove:
    #     print(f"\n⚠️  删除的输入特征 ({len(input_to_remove)}个):")
    #     for i, f in enumerate(input_to_remove, 1):
    #         if f in zero_stats_df['column_name'].values:
    #             zero_pct = zero_stats_df[zero_stats_df['column_name']==f]['zero_percentage'].values[0]
    #             print(f"   {i:2d}. {f:40s} (零值: {zero_pct:.1f}%)")
    #         else:
    #             print(f"   {i:2d}. {f}")

    # 执行删除
    print(f"\n⏳ 正在删除特征...")
    df_clean = df.drop(columns=features_to_remove_valid)

    # 更新特征列表
    input_features_clean = [f for f in input_features if f not in features_to_remove_valid]
    output_features_clean = [f for f in output_features if f not in features_to_remove_valid]

    # 统计报告
    print(f"\n✅ 删除完成！")
    print(f"\n📊 数据对比:")
    print(f"   {'':20s} {'删除前':>10s} {'删除后':>10s} {'变化':>10s}")
    print(f"   {'-'*50}")
    print(f"   {'总列数':20s} {len(df.columns):>10d} {len(df_clean.columns):>10d} {len(df_clean.columns)-len(df.columns):>10d}")
    print(f"   {'输入特征':20s} {len(input_features):>10d} {len(input_features_clean):>10d} {len(input_features_clean)-len(input_features):>10d}")
    print(f"   {'输出特征':20s} {len(output_features):>10d} {len(output_features_clean):>10d} {len(output_features_clean)-len(output_features):>10d}")
    print(f"   {'样本数':20s} {len(df):>10d} {len(df_clean):>10d} {0:>10d}")

    # 保存清洗后的特征列表
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # 保存为JSON
    import json

    feature_config = {
        'timestamp': timestamp,
        'original_counts': {
            'total_features': len(df.columns),
            'input_features': len(input_features),
            'output_features': len(output_features),
        },
        'cleaned_counts': {
            'total_features': len(df_clean.columns),
            'input_features': len(input_features_clean),
            'output_features': len(output_features_clean),
        },
        'removed_features': {
            'input': input_to_remove,
            'output': output_to_remove,
            'total': features_to_remove_valid,
        },
        'cleaned_features': {
            'input': input_features_clean,
            'output': output_features_clean,
        }
    }

    json_path = f'{save_dir}cleaned_features_{timestamp}.json'
    with open(json_path, 'w') as f:
        json.dump(feature_config, f, indent=2)

    print(f"\n💾 已保存特征配置: {json_path}")

    # 保存为TXT（方便复制）
    txt_path = f'{save_dir}cleaned_features_{timestamp}.txt'
    with open(txt_path, 'w') as f:
        f.write(f"=== 清洗后的特征列表 ===\n")
        f.write(f"时间: {timestamp}\n\n")

        f.write(f"输入特征 ({len(input_features_clean)}个):\n")
        for feat in input_features_clean:
            f.write(f"  {feat}\n")

        f.write(f"\n输出特征 ({len(output_features_clean)}个):\n")
        for feat in output_features_clean:
            f.write(f"  {feat}\n")

        f.write(f"\n删除的特征 ({len(features_to_remove_valid)}个):\n")
        for feat in features_to_remove_valid:
            f.write(f"  {feat}\n")

    print(f"💾 已保存特征列表: {txt_path}")

    # 生成删除报告
    removal_report = {
        'df_original_shape': df.shape,
        'df_clean_shape': df_clean.shape,
        'features_removed': len(features_to_remove_valid),
        'input_removed': len(input_to_remove),
        'output_removed': len(output_to_remove),
        'input_features_clean': input_features_clean,
        'output_features_clean': output_features_clean,
    }

    return df_clean, input_features_clean, output_features_clean, removal_report


In [29]:
# ============================================================================
# 运行Cell 2
# ============================================================================

# 确保INPUT_FEATURES和OUTPUT_FEATURES已定义
# INPUT_FEATURES = [col for col in df_train.columns if col not in OUTPUT_FEATURES]

# 应用删除
df_train_clean, INPUT_FEATURES_CLEAN, OUTPUT_FEATURES_CLEAN, report = apply_feature_removal(
    train_df,
    FEATURES_TO_REMOVE,
    INPUT_FEATURES,
    OUTPUT_FEATURES,
    save_dir='/content/'
)

# 显示最终结果
print("\n" + "="*80)
print("🎉 特征清洗完成！")
print("="*80)
print(f"\n📦 清洗后的数据:")
print(f"   变量名: df_train_clean")
print(f"   形状: {df_train_clean.shape}")
print(f"   输入特征: {len(INPUT_FEATURES_CLEAN)} 个 (INPUT_FEATURES_CLEAN)")
print(f"   输出特征: {len(OUTPUT_FEATURES_CLEAN)} 个 (OUTPUT_FEATURES_CLEAN)")

print(f"\n🚀 可以开始训练了！")
print(f"\n使用方法:")
print(f"   model, history = main_training(")
print(f"       df_train_clean,")
print(f"       INPUT_FEATURES_CLEAN,")
print(f"       OUTPUT_FEATURES_CLEAN")
print(f"   )")

# 可选：验证没有全零列
print(f"\n🔍 最终验证:")
all_zero_outputs = []
for col in OUTPUT_FEATURES_CLEAN:
    if (df_train_clean[col] == 0).all():
        all_zero_outputs.append(col)

if all_zero_outputs:
    print(f"   ⚠️  警告：仍有 {len(all_zero_outputs)} 个全零输出特征！")
    print(f"   {all_zero_outputs}")
else:
    print(f"   ✅ 没有全零输出特征")

# 验证零值高的输出特征
high_zero_outputs = []
for col in OUTPUT_FEATURES_CLEAN:
    zero_ratio = (df_train_clean[col] == 0).sum() / len(df_train_clean)
    if zero_ratio > 0.90:
        high_zero_outputs.append((col, zero_ratio))

if high_zero_outputs:
    print(f"   ⚠️  注意：有 {len(high_zero_outputs)} 个输出特征零值>90%")
    for col, ratio in high_zero_outputs[:5]:
        print(f"      - {col}: {ratio*100:.1f}%")
else:
    print(f"   ✅ 所有输出特征零值<90%")

print("\n" + "="*80)

📝 Cell 2: 应用特征删除

📊 删除统计:
   总删除: 138 个特征
   - 输入特征: 0 个
   - 输出特征: 138 个

⏳ 正在删除特征...

✅ 删除完成！

📊 数据对比:
                               删除前        删除后         变化
   --------------------------------------------------
   总列数                         793        655       -138
   输入特征                        490        490          0
   输出特征                        302        164       -138
   样本数                      625000     625000          0

💾 已保存特征配置: /content/cleaned_features_20251028_030207.json
💾 已保存特征列表: /content/cleaned_features_20251028_030207.txt

🎉 特征清洗完成！

📦 清洗后的数据:
   变量名: df_train_clean
   形状: (625000, 655)
   输入特征: 490 个 (INPUT_FEATURES_CLEAN)
   输出特征: 164 个 (OUTPUT_FEATURES_CLEAN)

🚀 可以开始训练了！

使用方法:
   model, history = main_training(
       df_train_clean,
       INPUT_FEATURES_CLEAN,
       OUTPUT_FEATURES_CLEAN
   )

🔍 最终验证:
   ✅ 没有全零输出特征
   ✅ 所有输出特征零值<90%



In [20]:
"""
================================================================================
Two-Stage Boost Transformer - 终极修复版 + R²诊断
================================================================================
⭐⭐⭐ 新增内容：
1. 多输出R²正确计算（逐信号计算后平均）
2. 添加R²诊断函数，检测异常情况
3. 在所有R²计算处使用修复后的函数

问题分析：
- sklearn的r2_score对多输出默认行为可能导致异常
- 如果某些输出方差极小，会导致R²异常
- 解决方案：逐信号计算R²，然后取平均或中位数

================================================================================
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.cuda.amp import autocast, GradScaler
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 Two-Stage Boost Transformer - 终极修复版 + R²诊断")
print("="*80)
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
print("="*80)

# ============================================================================
# 配置类
# ============================================================================
class Config:
    """全局配置类"""
    TRAIN_TEST_SPLIT = 0.15
    RANDOM_SEED = 42

    # Stage 1配置
    STAGE1_D_MODEL = 256
    STAGE1_NHEAD = 16
    STAGE1_NUM_LAYERS = 6
    STAGE1_DROPOUT = 0.1
    STAGE1_EPOCHS = 50
    STAGE1_LR = 0.0001
    STAGE1_BATCH_SIZE = 512
    STAGE1_WEIGHT_DECAY = 1e-5
    STAGE1_SCHEDULER_PATIENCE = 3
    STAGE1_SCHEDULER_FACTOR = 0.5
    STAGE1_GRAD_CLIP_NORM = 1.0

    # Stage 2配置
    # STAGE2_D_MODEL = 128
    # STAGE2_NHEAD = 8
    # STAGE2_NUM_LAYERS = 4
    STAGE2_D_MODEL = 256
    STAGE2_NHEAD = 16
    STAGE2_NUM_LAYERS = 6
    STAGE2_DROPOUT = 0.15
    STAGE2_EPOCHS = 80
    STAGE2_LR = 0.0001
    STAGE2_BATCH_SIZE = 512
    STAGE2_WEIGHT_DECAY = 5e-6
    STAGE2_SCHEDULER_PATIENCE = 15
    STAGE2_SCHEDULER_FACTOR = 0.7
    STAGE2_GRAD_CLIP_NORM = 0.5

    # Boosting配置
    R2_THRESHOLD = 0.4
    USE_SELECTIVE_BOOSTING = True

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    NUM_WORKERS = 0

config = Config()

print(f"\n⚙️  配置: 设备={config.DEVICE}")
print(f"Stage 1: {config.STAGE1_EPOCHS} epochs, lr={config.STAGE1_LR}")
print(f"Stage 2: {config.STAGE2_EPOCHS} epochs, lr={config.STAGE2_LR}")
print("="*80)

# ============================================================================
# ⭐⭐⭐ R²计算修复函数 ⭐⭐⭐
# ============================================================================
def compute_r2_safe(y_true, y_pred, method='per_output_mean'):
    """
    安全的R²计算函数 - 避免多输出时的异常值

    Args:
        y_true: 真实值 (n_samples, n_outputs)
        y_pred: 预测值 (n_samples, n_outputs)
        method: 计算方法
            - 'per_output_mean': 逐输出计算R²然后取平均（推荐）
            - 'per_output_median': 逐输出计算R²然后取中位数
            - 'sklearn_default': 使用sklearn默认方法
            - 'global': 全局R²（将所有输出flatten）

    Returns:
        r2: R²值
        per_output_r2: 每个输出的R²数组（用于诊断）
    """
    if y_true.ndim == 1:
        y_true = y_true.reshape(-1, 1)
        y_pred = y_pred.reshape(-1, 1)

    n_outputs = y_true.shape[1]
    per_output_r2 = np.zeros(n_outputs)

    # 逐输出计算R²
    for i in range(n_outputs):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]

        # 检查方差
        var_true = np.var(y_t)
        if var_true < 1e-10:
            print(f"  ⚠️  输出 {i}: 方差过小 ({var_true:.2e})，R²设为0")
            per_output_r2[i] = 0.0
        else:
            try:
                per_output_r2[i] = r2_score(y_t, y_p)
            except Exception as e:
                print(f"  ⚠️  输出 {i}: R²计算失败 ({e})，设为-1")
                per_output_r2[i] = -1.0

    # 根据method汇总
    if method == 'per_output_mean':
        # 过滤异常值后取平均
        valid_r2 = per_output_r2[np.isfinite(per_output_r2) & (per_output_r2 > -10)]
        if len(valid_r2) == 0:
            r2 = -1.0
        else:
            r2 = np.mean(valid_r2)
    elif method == 'per_output_median':
        valid_r2 = per_output_r2[np.isfinite(per_output_r2) & (per_output_r2 > -10)]
        if len(valid_r2) == 0:
            r2 = -1.0
        else:
            r2 = np.median(valid_r2)
    elif method == 'sklearn_default':
        r2 = r2_score(y_true, y_pred)
    elif method == 'global':
        r2 = r2_score(y_true.flatten(), y_pred.flatten())
    else:
        raise ValueError(f"Unknown method: {method}")

    return r2, per_output_r2


def diagnose_r2_computation(y_true, y_pred, name=""):
    """
    诊断R²计算 - 检测异常情况

    Args:
        y_true: 真实值
        y_pred: 预测值
        name: 数据名称
    """
    print("\n" + "="*80)
    print(f"🔍 R²计算诊断 - {name}")
    print("="*80)

    print(f"\n📊 数据统计:")
    print(f"  形状: {y_true.shape}")
    print(f"  y_true范围: [{np.min(y_true):.4f}, {np.max(y_true):.4f}]")
    print(f"  y_pred范围: [{np.min(y_pred):.4f}, {np.max(y_pred):.4f}]")
    print(f"  y_true均值: {np.mean(y_true):.4f}, 标准差: {np.std(y_true):.4f}")
    print(f"  y_pred均值: {np.mean(y_pred):.4f}, 标准差: {np.std(y_pred):.4f}")

    # 检查方差
    per_output_var = np.var(y_true, axis=0)
    low_var_outputs = np.where(per_output_var < 1e-6)[0]
    if len(low_var_outputs) > 0:
        print(f"\n  ⚠️  发现{len(low_var_outputs)}个低方差输出（方差<1e-6）")
        print(f"     索引: {low_var_outputs[:10]}...")

    # 计算不同方法的R²
    print(f"\n📊 不同方法的R²:")
    r2_mean, per_r2 = compute_r2_safe(y_true, y_pred, 'per_output_mean')
    print(f"  逐输出平均: {r2_mean:.6f}")

    r2_median, _ = compute_r2_safe(y_true, y_pred, 'per_output_median')
    print(f"  逐输出中位数: {r2_median:.6f}")

    try:
        r2_sklearn = r2_score(y_true, y_pred)
        print(f"  sklearn默认: {r2_sklearn:.6f}")
    except Exception as e:
        print(f"  sklearn默认: 计算失败 ({e})")

    r2_global, _ = compute_r2_safe(y_true, y_pred, 'global')
    print(f"  全局R²: {r2_global:.6f}")

    # 显示异常的输出
    abnormal_r2 = np.where((per_r2 < -10) | (per_r2 > 2))[0]
    if len(abnormal_r2) > 0:
        print(f"\n  🚨 发现{len(abnormal_r2)}个异常R²（<-10或>2）")
        print(f"     索引: {abnormal_r2[:10]}...")
        print(f"     R²值: {per_r2[abnormal_r2[:10]]}")

    # R²分布
    print(f"\n📊 R²分布:")
    print(f"  最小: {np.min(per_r2):.4f}")
    print(f"  Q1: {np.percentile(per_r2, 25):.4f}")
    print(f"  中位数: {np.median(per_r2):.4f}")
    print(f"  Q3: {np.percentile(per_r2, 75):.4f}")
    print(f"  最大: {np.max(per_r2):.4f}")

    print("="*80)

    return r2_mean, per_r2


# ============================================================================
# 模型定义
# ============================================================================
class CompactSensorTransformer(nn.Module):
    """紧凑型传感器Transformer"""
    def __init__(self, num_inputs, num_outputs, d_model=128, nhead=8,
                 num_layers=3, dropout=0.1, model_name="SST"):
        super().__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.d_model = d_model
        self.model_name = model_name

        self.input_embedding = nn.Linear(1, d_model)
        self.position_encoding = nn.Parameter(torch.randn(num_inputs, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*2,
            dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.output_projection = nn.Linear(d_model, num_outputs)

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.input_embedding(x) + self.position_encoding.unsqueeze(0)
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x = self.global_pool(x).squeeze(-1)
        return self.output_projection(x)

# ============================================================================
# 训练函数 - 使用修复的R²计算
# ============================================================================
def train_epoch(model, loader, criterion, optimizer, scaler, device, grad_clip_norm):
    """训练一个epoch"""
    model.train()
    total_loss = 0
    pbar = tqdm(loader, desc='Training', leave=False)

    for X_batch, y_batch in pbar:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()

        with autocast():
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    return total_loss / len(loader)


def evaluate(model, loader, device, scaler_y=None, use_safe_r2=True):
    """
    评估模型 - ⭐ 使用安全的R²计算 ⭐

    Args:
        use_safe_r2: 是否使用安全的R²计算（推荐True）
    """
    model.eval()
    y_true_scaled_all, y_pred_scaled_all = [], []

    with torch.no_grad():
        for X_batch, y_batch in tqdm(loader, desc='Evaluating', leave=False):
            X_batch = X_batch.to(device)

            with autocast():
                y_pred = model(X_batch).cpu().numpy()

            y_true_scaled_all.append(y_batch.numpy())
            y_pred_scaled_all.append(y_pred)

    y_true_scaled = np.vstack(y_true_scaled_all)
    y_pred_scaled = np.vstack(y_pred_scaled_all)

    # 反标准化到原始空间
    if scaler_y is not None:
        y_true = scaler_y.inverse_transform(y_true_scaled)
        y_pred = scaler_y.inverse_transform(y_pred_scaled)
    else:
        y_true = y_true_scaled
        y_pred = y_pred_scaled

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # ⭐⭐⭐ 使用安全的R²计算 ⭐⭐⭐
    if use_safe_r2:
        r2, _ = compute_r2_safe(y_true, y_pred, method='per_output_mean')
    else:
        r2 = r2_score(y_true, y_pred)

    return {'mae': mae, 'rmse': rmse, 'r2': r2}, y_true, y_pred


def train_model(model, train_loader, val_loader, config_dict, scaler_y=None, stage_name="Stage"):
    """通用训练流程"""
    print("\n" + "="*80)
    print(f"🚀 开始训练 - {stage_name}")
    print("="*80)

    lr = config_dict['lr']
    epochs = config_dict['epochs']
    weight_decay = config_dict.get('weight_decay', 1e-5)
    scheduler_patience = config_dict.get('scheduler_patience', 3)
    scheduler_factor = config_dict.get('scheduler_factor', 0.5)
    grad_clip_norm = config_dict.get('grad_clip_norm', 1.0)

    print(f"\n📋 训练配置:")
    print(f"  学习率: {lr}, 权重衰减: {weight_decay}")
    print(f"  训练轮数: {epochs}, 梯度裁剪: {grad_clip_norm}")
    print(f"  评估空间: {'原始空间' if scaler_y is not None else '标准化空间'}")
    print(f"  R²计算: 安全模式（逐输出平均）")

    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=scheduler_patience, factor=scheduler_factor
    )
    scaler = GradScaler()

    history = {
        'train_loss': [], 'val_mae': [], 'val_rmse': [], 'val_r2': [], 'lr': []
    }

    best_mae = float('inf')
    best_r2 = -float('inf')
    best_epoch = 0

    for epoch in range(epochs):
        print(f"\n📍 Epoch {epoch+1}/{epochs}")
        print("-" * 60)

        train_loss = train_epoch(
            model, train_loader, criterion, optimizer, scaler,
            config.DEVICE, grad_clip_norm
        )

        # ⭐ 使用安全的R²计算
        val_metrics, _, _ = evaluate(model, val_loader, config.DEVICE, scaler_y=scaler_y, use_safe_r2=True)

        scheduler.step(val_metrics['mae'])
        current_lr = optimizer.param_groups[0]['lr']

        history['train_loss'].append(train_loss)
        history['val_mae'].append(val_metrics['mae'])
        history['val_rmse'].append(val_metrics['rmse'])
        history['val_r2'].append(val_metrics['r2'])
        history['lr'].append(current_lr)

        print(f"  训练损失: {train_loss:.6f}")
        print(f"  验证MAE:  {val_metrics['mae']:.6f}")
        print(f"  验证RMSE: {val_metrics['rmse']:.6f}")
        print(f"  验证R²:   {val_metrics['r2']:.6f}")
        print(f"  学习率:   {current_lr:.8f}")

        if val_metrics['r2'] < -10:
            print(f"  🚨 警告：R²异常低 ({val_metrics['r2']:.2f})！")

        if val_metrics['mae'] < best_mae:
            best_mae = val_metrics['mae']
            best_r2 = val_metrics['r2']
            best_epoch = epoch + 1
            print(f"  ✅ 最佳模型已更新！")

    print("\n" + "="*80)
    print(f"✅ {stage_name} 训练完成！")
    print(f"   最佳验证MAE: {best_mae:.6f} | R²: {best_r2:.6f} (Epoch {best_epoch})")
    print("="*80)

    return model, history, best_mae, best_r2


# ============================================================================
# 数据准备
# ============================================================================
def prepare_data(X, y, batch_size, standardize_y=True):
    """准备训练和验证数据"""
    print("\n" + "="*80)
    print("📊 准备训练数据")
    print("="*80)

    print(f"\n✅ 数据形状: X={X.shape}, y={y.shape}")

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=config.TRAIN_TEST_SPLIT, random_state=config.RANDOM_SEED
    )

    print(f"✅ 数据划分: 训练{X_train.shape[0]:,}, 验证{X_val.shape[0]:,}")

    scaler_X = StandardScaler()
    scaler_y = StandardScaler() if standardize_y else None

    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)

    if standardize_y:
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_val_scaled = scaler_y.transform(y_val)
        print(f"  ✓ y标准化完成")
    else:
        y_train_scaled = y_train
        y_val_scaled = y_val
        print("  ⚠️  跳过y标准化（残差模式）")

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train_scaled),
        torch.FloatTensor(y_train_scaled)
    )
    val_dataset = TensorDataset(
        torch.FloatTensor(X_val_scaled),
        torch.FloatTensor(y_val_scaled)
    )

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=config.NUM_WORKERS, pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=config.NUM_WORKERS, pin_memory=True
    )

    return train_loader, val_loader, scaler_X, scaler_y, X_train, X_val, y_train, y_val


# ============================================================================
# 辅助函数
# ============================================================================
def create_stage_config(stage=1):
    """创建Stage配置字典"""
    if stage == 1:
        return {
            'epochs': config.STAGE1_EPOCHS,
            'lr': config.STAGE1_LR,
            'weight_decay': config.STAGE1_WEIGHT_DECAY,
            'scheduler_patience': config.STAGE1_SCHEDULER_PATIENCE,
            'scheduler_factor': config.STAGE1_SCHEDULER_FACTOR,
            'grad_clip_norm': config.STAGE1_GRAD_CLIP_NORM
        }
    elif stage == 2:
        return {
            'epochs': config.STAGE2_EPOCHS,
            'lr': config.STAGE2_LR,
            'weight_decay': config.STAGE2_WEIGHT_DECAY,
            'scheduler_patience': config.STAGE2_SCHEDULER_PATIENCE,
            'scheduler_factor': config.STAGE2_SCHEDULER_FACTOR,
            'grad_clip_norm': config.STAGE2_GRAD_CLIP_NORM
        }
    else:
        raise ValueError(f"stage must be 1 or 2, got {stage}")


def print_training_config_comparison():
    """打印配置对比"""
    print("\n" + "="*80)
    print("📊 Stage 1 vs Stage 2 配置对比")
    print("="*80)

    stage1 = create_stage_config(1)
    stage2 = create_stage_config(2)

    print(f"\n{'参数':<25} {'Stage 1':<20} {'Stage 2':<20}")
    print("-" * 80)
    for key in stage1.keys():
        print(f"{key:<25} {stage1[key]:<20} {stage2[key]:<20}")
    print("="*80)


# ============================================================================
# 可视化（简化版）
# ============================================================================
def plot_training_history(history_stage1, history_stage2):
    """绘制训练历史"""
    metrics = ['train_loss', 'val_mae', 'val_r2']
    titles = ['Training Loss', 'Validation MAE', 'Validation R²']

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        ax = axes[idx]

        if metric in history_stage1:
            ax.plot(history_stage1[metric], label='Stage 1', linewidth=2)

        if metric in history_stage2:
            stage2_x = np.arange(len(history_stage1[metric]),
                                len(history_stage1[metric]) + len(history_stage2[metric]))
            ax.plot(stage2_x, history_stage2[metric], label='Stage 2', linewidth=2, color='orange')

        ax.set_xlabel('Epoch')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.legend()
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
    print("\n📊 训练历史已保存: training_history.png")
    plt.close()


# ============================================================================
# Boost训练流程 - 使用修复的R²计算
# ============================================================================
def boost_training_pipeline(X, y, input_features, output_features, enable_r2_diagnosis=True):
    """
    Two-Stage Boosting完整训练流程

    Args:
        enable_r2_diagnosis: 是否启用R²诊断（默认True）
    """
    print("\n" + "="*80)
    print("🏗️  Two-Stage Boosting Training Pipeline")
    print("="*80)

    # Stage 1
    print("\n" + "="*80)
    print("📊 STAGE 1: Base Model Training")
    print("="*80)

    train_loader_s1, val_loader_s1, scaler_X_s1, scaler_y_s1, X_train_s1, X_val_s1, y_train_s1, y_val_s1 = \
        prepare_data(X, y, config.STAGE1_BATCH_SIZE, standardize_y=True)

    base_model = CompactSensorTransformer(
        num_inputs=X.shape[1],
        num_outputs=y.shape[1],
        d_model=config.STAGE1_D_MODEL,
        nhead=config.STAGE1_NHEAD,
        num_layers=config.STAGE1_NUM_LAYERS,
        dropout=config.STAGE1_DROPOUT,
        model_name="Stage1_Base"
    ).to(config.DEVICE)

    print(f"\n📊 Base Model: {sum(p.numel() for p in base_model.parameters()):,} 参数")

    stage1_config = create_stage_config(stage=1)

    base_model, history_s1, best_mae_s1, best_r2_s1 = train_model(
        base_model, train_loader_s1, val_loader_s1, stage1_config,
        scaler_y=scaler_y_s1,
        stage_name="Stage 1: Base Model"
    )

    torch.save(base_model.state_dict(), 'boost_stage1_best.pth')
    with open('stage1_scalers.pkl', 'wb') as f:
        pickle.dump({'scaler_X': scaler_X_s1, 'scaler_y': scaler_y_s1}, f)
    print(f"\n💾 Stage 1模型和scalers已保存")

    # 计算残差
    print("\n" + "="*80)
    print("📊 计算Stage 1残差")
    print("="*80)

    base_model.eval()

    X_train_s1_scaled = scaler_X_s1.transform(X_train_s1)
    X_train_s1_tensor = torch.FloatTensor(X_train_s1_scaled).to(config.DEVICE)

    with torch.no_grad():
        y_train_s1_pred_scaled = base_model(X_train_s1_tensor).cpu().numpy()

    y_train_s1_pred = scaler_y_s1.inverse_transform(y_train_s1_pred_scaled)
    residual_after_stage1 = y_train_s1 - y_train_s1_pred

    print(f"✅ 残差统计:")
    print(f"  均值: {np.mean(residual_after_stage1):.6f}")
    print(f"  标准差: {np.std(residual_after_stage1):.6f}")

    # Stage 2
    print("\n" + "="*80)
    print("📊 STAGE 2: Residual Model Training")
    print("="*80)

    train_loader_s2, val_loader_s2, scaler_X_s2, scaler_residual, _, _, _, _ = \
        prepare_data(
            X[:len(X_train_s1)],
            residual_after_stage1,
            config.STAGE2_BATCH_SIZE,
            standardize_y=True
        )

    residual_model = CompactSensorTransformer(
        num_inputs=X.shape[1],
        num_outputs=y.shape[1],
        d_model=config.STAGE2_D_MODEL,
        nhead=config.STAGE2_NHEAD,
        num_layers=config.STAGE2_NUM_LAYERS,
        dropout=config.STAGE2_DROPOUT,
        model_name="Stage2_Residual"
    ).to(config.DEVICE)

    print(f"\n📊 Residual Model: {sum(p.numel() for p in residual_model.parameters()):,} 参数")

    stage2_config = create_stage_config(stage=2)

    residual_model, history_s2, best_mae_s2, best_r2_s2 = train_model(
        residual_model, train_loader_s2, val_loader_s2, stage2_config,
        scaler_y=scaler_residual,
        stage_name="Stage 2: Residual Model"
    )

    torch.save(residual_model.state_dict(), 'boost_stage2_best.pth')
    with open('stage2_scalers.pkl', 'wb') as f:
        pickle.dump({'scaler_X': scaler_X_s2, 'scaler_residual': scaler_residual}, f)
    print(f"\n💾 Stage 2模型和scalers已保存")

    # 验证评估
    print("\n" + "="*80)
    print("📊 验证集评估")
    print("="*80)

    base_model.eval()
    residual_model.eval()

    X_val_s1_scaled = scaler_X_s1.transform(X_val_s1)
    X_val_s1_tensor = torch.FloatTensor(X_val_s1_scaled).to(config.DEVICE)

    with torch.no_grad():
        y_val_s1_pred_scaled = base_model(X_val_s1_tensor).cpu().numpy()

    y_val_s1_pred = scaler_y_s1.inverse_transform(y_val_s1_pred_scaled)

    # ⭐ R²诊断
    if enable_r2_diagnosis:
        stage1_r2, signal_r2_scores = diagnose_r2_computation(y_val_s1, y_val_s1_pred, "Stage 1验证集")
    else:
        stage1_r2, signal_r2_scores = compute_r2_safe(y_val_s1, y_val_s1_pred)

    X_val_s2_scaled = scaler_X_s2.transform(X_val_s1)
    X_val_s2_tensor = torch.FloatTensor(X_val_s2_scaled).to(config.DEVICE)

    with torch.no_grad():
        residual_val_pred_scaled = residual_model(X_val_s2_tensor).cpu().numpy()

    residual_val_pred = scaler_residual.inverse_transform(residual_val_pred_scaled)

    y_val_boosted_all = y_val_s1_pred + residual_val_pred

    if config.USE_SELECTIVE_BOOSTING:
        weak_signals_mask = signal_r2_scores < config.R2_THRESHOLD
        num_weak = np.sum(weak_signals_mask)
        print(f"\n⭐ 选择性Boosting: {num_weak}/{len(signal_r2_scores)} 信号需要Boosting")

        y_val_selective_boosted = y_val_s1_pred.copy()
        y_val_selective_boosted[:, weak_signals_mask] = y_val_boosted_all[:, weak_signals_mask]
    else:
        y_val_selective_boosted = y_val_boosted_all
        weak_signals_mask = np.ones(len(signal_r2_scores), dtype=bool)

    # 最终评估
    print("\n" + "="*80)
    print("📊 最终评估")
    print("="*80)

    stage1_mae = mean_absolute_error(y_val_s1, y_val_s1_pred)

    selective_boosted_mae = mean_absolute_error(y_val_s1, y_val_selective_boosted)
    selective_boosted_r2, _ = compute_r2_safe(y_val_s1, y_val_selective_boosted)

    improvement = (stage1_mae - selective_boosted_mae) / stage1_mae * 100

    print(f"\n{'指标':<20} {'Stage 1':<18} {'Selective Boost':<18}")
    print("-" * 60)
    print(f"{'MAE':<20} {stage1_mae:<18.6f} {selective_boosted_mae:<18.6f}")
    print(f"{'R²':<20} {stage1_r2:<18.6f} {selective_boosted_r2:<18.6f}")
    print(f"{'MAE改进':<20} {'Baseline':<18} {improvement:>+16.2f}%")
    print("="*80)

    if improvement > 0:
        print(f"\n✅ Boosting有效！MAE改进 {improvement:.2f}%")

    plot_training_history(history_s1, history_s2)

    scalers = {
        'stage1_scaler_X': scaler_X_s1,
        'stage1_scaler_y': scaler_y_s1,
        'stage2_scaler_X': scaler_X_s2,
        'stage2_scaler_residual': scaler_residual
    }

    history = {'stage1': history_s1, 'stage2': history_s2}

    results = {
        'stage1_mae': stage1_mae,
        'stage1_r2': stage1_r2,
        'selective_boosted_mae': selective_boosted_mae,
        'selective_boosted_r2': selective_boosted_r2,
        'improvement_selective': improvement,
        'signal_r2_scores': signal_r2_scores,
        'weak_signals_mask': weak_signals_mask
    }

    with open('boost_results.pkl', 'wb') as f:
        pickle.dump({'scalers': scalers, 'history': history, 'results': results}, f)
    print(f"\n💾 结果已保存: boost_results.pkl")

    return base_model, residual_model, scalers, history, results, None


# ============================================================================
# Boost推理 - 使用修复的R²
# ============================================================================
def boost_inference(X_new, base_model, residual_model, scalers,
                   signal_r2_scores=None, r2_threshold=None, device=None):
    """Boost推理"""
    if device is None:
        device = config.DEVICE

    if r2_threshold is None:
        r2_threshold = config.R2_THRESHOLD

    base_model.eval()
    residual_model.eval()

    scaler_X_s1 = scalers['stage1_scaler_X']
    scaler_y_s1 = scalers['stage1_scaler_y']

    X_scaled_s1 = scaler_X_s1.transform(X_new)
    X_tensor_s1 = torch.FloatTensor(X_scaled_s1).to(device)

    with torch.no_grad():
        with autocast():
            y_pred_s1_scaled = base_model(X_tensor_s1).cpu().numpy()

    y_pred_s1 = scaler_y_s1.inverse_transform(y_pred_s1_scaled)

    scaler_X_s2 = scalers['stage2_scaler_X']
    scaler_residual = scalers['stage2_scaler_residual']

    X_scaled_s2 = scaler_X_s2.transform(X_new)
    X_tensor_s2 = torch.FloatTensor(X_scaled_s2).to(device)

    with torch.no_grad():
        with autocast():
            residual_pred_scaled = residual_model(X_tensor_s2).cpu().numpy()

    residual_pred = scaler_residual.inverse_transform(residual_pred_scaled)

    y_pred_full = y_pred_s1 + residual_pred

    if config.USE_SELECTIVE_BOOSTING and signal_r2_scores is not None:
        mask = signal_r2_scores < r2_threshold
        y_pred = y_pred_s1.copy()
        y_pred[:, mask] = y_pred_full[:, mask]
    else:
        y_pred = y_pred_full
        mask = np.ones(y_pred_s1.shape[1], dtype=bool)

    return y_pred, mask


# ============================================================================
# 使用提示
# ============================================================================
print("\n" + "="*80)
print("✅ 代码加载完成！")
print("="*80)
print("\n⭐⭐⭐ 本版本特色：")
print("  ✓ 安全的R²计算（逐输出平均，避免异常值）")
print("  ✓ R²诊断函数（检测数据和计算问题）")
print("  ✓ evaluate函数在原始空间评估")
print("  ✓ Stage 2残差正确反标准化")
print("\n📋 主要函数:")
print("  • diagnose_r2_computation() - R²诊断")
print("  • compute_r2_safe() - 安全R²计算")
print("  • boost_training_pipeline() - 完整训练")
print("  • boost_inference() - 推理")
print("="*80)

print("\n🎉 准备就绪！R²计算已完全修复！")

🚀 Two-Stage Boost Transformer - 终极修复版 + R²诊断
PyTorch版本: 2.8.0+cu126
CUDA可用: True

⚙️  配置: 设备=cuda
Stage 1: 50 epochs, lr=0.0001
Stage 2: 80 epochs, lr=0.0001

✅ 代码加载完成！

⭐⭐⭐ 本版本特色：
  ✓ 安全的R²计算（逐输出平均，避免异常值）
  ✓ R²诊断函数（检测数据和计算问题）
  ✓ evaluate函数在原始空间评估
  ✓ Stage 2残差正确反标准化

📋 主要函数:
  • diagnose_r2_computation() - R²诊断
  • compute_r2_safe() - 安全R²计算
  • boost_training_pipeline() - 完整训练
  • boost_inference() - 推理

🎉 准备就绪！R²计算已完全修复！


In [21]:
X = df_train_clean[INPUT_FEATURES_CLEAN].values
y = df_train_clean[OUTPUT_FEATURES_CLEAN].values
input_features = INPUT_FEATURES_CLEAN
output_features = OUTPUT_FEATURES_CLEAN
base_model, residual_model, scalers, history, results, _ = \
    boost_training_pipeline(X, y, input_features, output_features)


🏗️  Two-Stage Boosting Training Pipeline

📊 STAGE 1: Base Model Training

📊 准备训练数据

✅ 数据形状: X=(312500, 490), y=(312500, 164)
✅ 数据划分: 训练265,625, 验证46,875
  ✓ y标准化完成

📊 Base Model: 3,330,724 参数

🚀 开始训练 - Stage 1: Base Model

📋 训练配置:
  学习率: 0.0001, 权重衰减: 1e-05
  训练轮数: 50, 梯度裁剪: 1.0
  评估空间: 原始空间
  R²计算: 安全模式（逐输出平均）

📍 Epoch 1/50
------------------------------------------------------------


Training:   0%|          | 0/519 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
"""
从已保存的Base Model继续完成Stage 2训练
前提：boost_stage1_best.pth 和 stage1_scalers.pkl 已存在
"""

# Ensure necessary functions and Config are defined (assuming they are in W7y9yU-VbkTD)
# from W7y9yU-VbkTD import CompactSensorTransformer, train_model, prepare_data, compute_residuals_correctly, create_stage_config, config, diagnose_r2_computation, compute_r2_safe, plot_training_history

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.cuda.amp import autocast, GradScaler
import warnings
warnings.filterwarnings('ignore')


print("\n" + "="*80)
print("🚀 Stage 2 训练 (基于已保存的Stage 1)")
print("="*80)

# ============================================================================
# STEP 1: 加载Base Model和Scalers
# ============================================================================

print("\n" + "="*80)
print("📥 加载已保存的Base Model和Scalers")
print("="*80)

# Load Stage 1 model state dict
try:
    base_model_state_dict = torch.load('/content/boost_stage1_best.pth', map_location=config.DEVICE)
    print("✅ Stage 1 Model State Dict加载成功")
except FileNotFoundError:
    print("❌ 错误: /content/boost_stage1_best.pth 文件未找到！")
    raise
except Exception as e:
    print(f"❌ 错误: 加载 /content/boost_stage1_best.pth 时发生异常: {e}")
    raise


# Load Stage 1 Scalers and metrics
try:
    with open('/content/stage1_scalers.pkl', 'rb') as f:
        scalers_s1_info = pickle.load(f)
    scaler_X_s1 = scalers_s1_info['scaler_X']
    scaler_y_s1 = scalers_s1_info['scaler_y']
    # Assume num_inputs and num_outputs were saved with scalers or can be inferred
    # If not saved, you might need to load training_info.pkl or infer from data
    num_inputs = X.shape[1] # Infer from data if not in scaler file
    num_outputs = y.shape[1] # Infer from data if not in scaler file

    # Try loading best_mae and best_r2 if they were saved with scalers
    best_mae_s1 = scalers_s1_info.get('best_mae', float('inf'))
    best_r2_s1 = scalers_s1_info.get('best_r2', -float('inf'))

    print("✅ Stage 1 Scalers加载成功")
    print(f"   最佳MAE (from scalers): {best_mae_s1:.6f}")
    print(f"   最佳R² (from scalers): {best_r2_s1:.6f}")

except FileNotFoundError:
    print("❌ 错误: /content/stage1_scalers.pkl 文件未找到！")
    raise
except Exception as e:
    print(f"❌ 错误: 加载 /content/stage1_scalers.pkl 时发生异常: {e}")
    raise

# Reconstruct Base Model
# Ensure CompactSensorTransformer class is defined (e.g., in cell W7y9yU-VbkTD)
base_model = CompactSensorTransformer(
    num_inputs=num_inputs,
    num_outputs=num_outputs,
    d_model=config.STAGE1_D_MODEL,
    nhead=config.STAGE1_NHEAD,
    num_layers=config.STAGE1_NUM_LAYERS,
    dropout=config.STAGE1_DROPOUT,
    model_name="BaseModel"
).to(config.DEVICE)

base_model.load_state_dict(base_model_state_dict)
base_model.eval()

print("✅ Base Model已重建并设置为评估模式")

# ============================================================================
# STEP 2: 准备原始数据（从cell5加载）
# ============================================================================

print("\n" + "="*80)
print("📥 加载原始训练数据")
print("="*80)

# ⭐ 从cell5中加载原始数据 (Assuming these variables are available in the environment)
try:
    # Check if df_train_clean, INPUT_FEATURES_CLEAN, OUTPUT_FEATURES_CLEAN are defined
    _ = df_train_clean
    _ = INPUT_FEATURES_CLEAN
    _ = OUTPUT_FEATURES_CLEAN

    X = df_train_clean[INPUT_FEATURES_CLEAN].values
    y = df_train_clean[OUTPUT_FEATURES_CLEAN].values


    print(f"✅ 数据加载成功 (from environment):")
    print(f"   输入形状: {X.shape}")
    print(f"   输出形状: {y.shape}")

except NameError:
    print("❌ 错误: 无法找到原始数据变量 (df_train_clean, INPUT_FEATURES_CLEAN, OUTPUT_FEATURES_CLEAN)")
    print("   请确保cell5已运行并定义了这些变量。")
    raise

# Split data into train and validation sets using the same seed as Stage 1
# This split will be used to calculate residuals on the training portion
# and evaluate Stage 2 on the validation portion.
X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(
    X, y, test_size=config.TRAIN_TEST_SPLIT, random_state=config.RANDOM_SEED
)
print(f"✅ 原始数据划分: 训练集 {X_train_orig.shape[0]:,}, 验证集 {X_val_orig.shape[0]:,}")


# ============================================================================
# STEP 3: 计算Stage 1残差
# ============================================================================

# Ensure compute_residuals_correctly is defined (e.g., in cell W7y9yU-VbkTD)
# If not defined, manually implement the logic here:
def compute_residuals_correctly(X_orig, y_orig, base_model, scaler_X, scaler_y, device):
    """
    Correctly compute residuals in original scale.
    """
    print("\n" + "="*80)
    print("🔧 计算Stage 2残差（正确方法）")
    print("="*80)

    base_model.eval()

    print(f"\n步骤1: 标准化输入")
    X_scaled = scaler_X.transform(X_orig)

    print(f"\n步骤2: Base模型预测（标准化空间）")
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        # ⭐ Batch processing to avoid OOM
        batch_size = 1024 # Can adjust this batch size
        y_pred_scaled_list = []
        for i in tqdm(range(0, len(X_tensor), batch_size), desc="Predicting Base Model Residuals"):
            batch = X_tensor[i:i+batch_size]
            # Ensure model forward pass uses autocast if needed and is compatible
            with autocast(): # Assuming autocast is available and needed
                 y_pred_batch = base_model(batch).cpu().numpy()
            y_pred_scaled_list.append(y_pred_batch)
        y_pred_scaled = np.vstack(y_pred_scaled_list)


    print(f"  预测形状: {y_pred_scaled.shape}")
    print(f"  预测范围: [{np.min(y_pred_scaled):.6f}, {np.max(y_pred_scaled):.6f}]")

    print(f"\n步骤3: 反标准化到原始空间")
    y_pred_original = scaler_y.inverse_transform(y_pred_scaled)
    print(f"  反标准化后范围: [{np.min(y_pred_original):.6f}, {np.max(y_pred_original):.6f}]")
    print(f"  真实值范围: [{np.min(y_orig):.6f}, {np.max(y_orig):.6f}]")


    print(f"\n步骤4: 计算残差（原始空间）")
    residuals = y_orig - y_pred_original
    print(f"  残差形状: {residuals.shape}")
    print(f"  残差范围: [{np.min(residuals):.6f}, {np.max(residuals):.6f}]")
    print(f"  残差均值: {np.mean(residuals):.6f} (应接近0)")
    print(f"  残差标准差: {np.std(residuals):.6f}")

    # Verification
    residual_ratio = np.std(residuals) / np.std(y_orig)
    print(f"\n验证:")
    print(f"  残差标准差/目标标准差 = {residual_ratio:.4f}")
    if residual_ratio < 0.3:
        print(f"  ✅ Base model performed well, residuals are small.")
    elif residual_ratio > 0.8:
         print(f"  ⚠️  Base model performed poorly, residuals are large.")
    else:
        print(f"  ✓ Base model performed moderately.")

    print("="*80)

    return residuals


print("\n" + "="*80)
print("📊 计算Stage 1残差")
print("="*80)

# Compute residuals on the training data subset
residuals_train = compute_residuals_correctly(
    X_train_orig, y_train_orig, base_model, scaler_X_s1, scaler_y_s1, config.DEVICE
)

# ============================================================================
# STEP 4: 准备Stage 2训练数据
# ============================================================================

print("\n" + "="*80)
print("📊 STEP 4: 准备Stage 2训练数据")
print("="*80)

# Stage 2 input is the same as Stage 1 input (standardized by a new scaler)
# Stage 2 target is the residual from Stage 1 (not standardized initially, but will be by Stage 2 scaler)

# Prepare data for Stage 2 training (X_train_orig -> residuals_train)
# We need a new scaler for X for Stage 2, even if it's the same original data
# We also need a scaler for the residuals
scaler_X_s2 = StandardScaler()
scaler_residual = StandardScaler()

# Standardize X_train_orig and residuals_train for Stage 2 training
X_train_s2_scaled = scaler_X_s2.fit_transform(X_train_orig)
residual_train_scaled = scaler_residual.fit_transform(residuals_train)


# Standardize X_val_orig and residuals_val for Stage 2 validation
# Need to calculate residuals_val first
base_model.eval()
residuals_val = compute_residuals_correctly(
    X_val_orig, y_val_orig, base_model, scaler_X_s1, scaler_y_s1, config.DEVICE
)

X_val_s2_scaled = scaler_X_s2.transform(X_val_orig)
residual_val_scaled = scaler_residual.transform(residuals_val)


print(f"✅ Stage 2训练数据形状: X={X_train_s2_scaled.shape}, Residual={residual_train_scaled.shape}")
print(f"✅ Stage 2验证数据形状: X={X_val_s2_scaled.shape}, Residual={residual_val_scaled.shape}")

# Create Stage 2 DataLoaders
train_dataset_s2 = TensorDataset(
    torch.FloatTensor(X_train_s2_scaled),
    torch.FloatTensor(residual_train_scaled)
)
val_dataset_s2 = TensorDataset(
    torch.FloatTensor(X_val_s2_scaled),
    torch.FloatTensor(residual_val_scaled)
)

train_loader_s2 = DataLoader(
    train_dataset_s2,
    batch_size=config.STAGE2_BATCH_SIZE,
    shuffle=True,
    num_workers=config.NUM_WORKERS,
    pin_memory=True
)
val_loader_s2 = DataLoader(
    val_dataset_s2,
    batch_size=config.STAGE2_BATCH_SIZE,
    shuffle=False,
    num_workers=config.NUM_WORKERS,
    pin_memory=True
)

print(f"✅ Stage 2 DataLoaders创建完成")


# ============================================================================
# STEP 5: 初始化并训练Stage 2 Residual Model
# ============================================================================

print("\n" + "="*80)
print("🚀 STEP 5: 训练Stage 2 Residual Model")
print("="*80)

# Initialize Stage 2 model
# Ensure CompactSensorTransformer class is defined (e.g., in cell W7y9yU-VbkTD)
residual_model = CompactSensorTransformer(
    num_inputs=num_inputs,
    num_outputs=num_outputs,
    d_model=config.STAGE2_D_MODEL,
    nhead=config.STAGE2_NHEAD,
    num_layers=config.STAGE2_NUM_LAYERS,
    dropout=config.STAGE2_DROPOUT,
    model_name="Stage2_Residual"
).to(config.DEVICE)

print(f"\n📊 Residual Model: {sum(p.numel() for p in residual_model.parameters()):,} 参数")

# Train Stage 2 model
stage2_config = create_stage_config(stage=2) # Ensure create_stage_config is defined

# Ensure train_model function is defined (e.g., in cell W7y9yU-VbkTD)
# If not defined, manually implement the training loop using the evaluate function with scaler_y=scaler_residual
def train_model(model, train_loader, val_loader, config_dict, scaler_y=None, stage_name="Stage"):
    """通用训练流程 - Using corrected evaluate and train_epoch"""
    print("\n" + "="*80)
    print(f"🚀 开始训练 - {stage_name}")
    print("="*80)

    lr = config_dict['lr']
    epochs = config_dict['epochs']
    weight_decay = config_dict.get('weight_decay', 1e-5)
    scheduler_patience = config_dict.get('scheduler_patience', 3)
    scheduler_factor = config_dict.get('scheduler_factor', 0.5)
    grad_clip_norm = config_dict.get('grad_clip_norm', 1.0)

    print(f"\n📋 训练配置:")
    print(f"  学习率: {lr}, 权重衰减: {weight_decay}")
    print(f"  训练轮数: {epochs}, 梯度裁剪: {grad_clip_norm}")
    print(f"  评估空间: {'原始空间' if scaler_y is not None else '标准化空间'}")
    print(f"  R²计算: 安全模式（逐输出平均）")


    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=scheduler_patience, factor=scheduler_factor
    )
    scaler = GradScaler()

    history = {
        'train_loss': [], 'val_mae': [], 'val_rmse': [], 'val_r2': [], 'lr': []
    }

    best_mae = float('inf')
    best_r2 = -float('inf')
    best_epoch = 0

    for epoch in range(epochs):
        print(f"\n📍 Epoch {epoch+1}/{epochs}")
        print("-" * 60)

        # Ensure train_epoch is defined and accepts scaler and grad_clip_norm
        # train_epoch(model, loader, criterion, optimizer, scaler, device, grad_clip_norm)
        def train_epoch(model, loader, criterion, optimizer, scaler, device, grad_clip_norm):
            """训练一个epoch"""
            model.train()
            total_loss = 0
            pbar = tqdm(loader, desc='Training', leave=False)

            for X_batch, y_batch in pbar:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)

                optimizer.zero_grad()

                with autocast():
                    y_pred = model(X_batch)
                    loss = criterion(y_pred, y_batch)

                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
                scaler.step(optimizer)
                scaler.update()

                total_loss += loss.item()
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})

            return total_loss / len(loader)


        train_loss = train_epoch(
            model, train_loader, criterion, optimizer, scaler,
            config.DEVICE, grad_clip_norm
        )

        # Ensure evaluate function is defined and accepts scaler_y and use_safe_r2
        # evaluate(model, loader, device, scaler_y=None, use_safe_r2=True)
        def evaluate(model, loader, device, scaler_y=None, use_safe_r2=True):
            """
            评估模型 - ⭐ 使用安全的R²计算 ⭐
            """
            model.eval()
            y_true_scaled_all, y_pred_scaled_all = [], []

            with torch.no_grad():
                for X_batch, y_batch in tqdm(loader, desc='Evaluating', leave=False):
                    X_batch = X_batch.to(device)

                    with autocast():
                        y_pred = model(X_batch).cpu().numpy()

                    y_true_scaled_all.append(y_batch.numpy())
                    y_pred_scaled_all.append(y_pred)

            y_true_scaled = np.vstack(y_true_scaled_all)
            y_pred_scaled = np.vstack(y_pred_scaled_all)

            # Inverse transform to original scale
            if scaler_y is not None:
                y_true = scaler_y.inverse_transform(y_true_scaled)
                y_pred = scaler_y.inverse_transform(y_pred_scaled)
            else:
                y_true = y_true_scaled
                y_pred = y_pred_scaled

            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            # ⭐⭐⭐ Use safe R² calculation ⭐⭐⭐
            if use_safe_r2:
                r2, _ = compute_r2_safe(y_true, y_pred, method='per_output_mean') # Ensure compute_r2_safe is defined
            else:
                r2 = r2_score(y_true, y_pred)


            return {'mae': mae, 'rmse': rmse, 'r2': r2}, y_true, y_pred

        val_metrics, _, _ = evaluate(model, val_loader, config.DEVICE, scaler_y=scaler_y, use_safe_r2=True)


        scheduler.step(val_metrics['mae'])
        current_lr = optimizer.param_groups[0]['lr']

        history['train_loss'].append(train_loss)
        history['val_mae'].append(val_metrics['mae'])
        history['val_rmse'].append(val_metrics['rmse'])
        history['val_r2'].append(val_metrics['r2'])
        history['lr'].append(current_lr)

        print(f"  训练损失: {train_loss:.6f}")
        print(f"  验证MAE:  {val_metrics['mae']:.6f}")
        print(f"  验证RMSE: {val_metrics['rmse']:.6f}")
        print(f"  验证R²:   {val_metrics['r2']:.6f}")
        print(f"  学习率:   {current_lr:.8f}")

        if val_metrics['r2'] < -10:
            print(f"  🚨 警告：R²异常低 ({val_metrics['r2']:.2f})！")

        if val_metrics['mae'] < best_mae:
            best_mae = val_metrics['mae']
            best_r2 = val_metrics['r2']
            best_epoch = epoch + 1
            print(f"  ✅ 最佳模型已更新！")

    print("\n" + "="*80)
    print(f"✅ {stage_name} 训练完成！")
    print(f"   最佳验证MAE: {best_mae:.6f} | R²: {best_r2:.6f} (Epoch {best_epoch})")
    print("="*80)

    return model, history, best_mae, best_r2


residual_model, history_s2, best_mae_s2, best_r2_s2 = train_model(
    residual_model, train_loader_s2, val_loader_s2, stage2_config,
    scaler_y=scaler_residual, # ⭐ Pass residual scaler for evaluation
    stage_name="Stage 2: Residual Model"
)


# ============================================================================
# STEP 6: 保存Stage 2 Model and Scalers
# ============================================================================

print("\n" + "="*80)
print("💾 STEP 6: 保存Stage 2 Model and Scalers")
print("="*80)

torch.save(residual_model.state_dict(), '/content/boost_stage2_best.pth')
with open('/content/stage2_scalers.pkl', 'wb') as f:
    pickle.dump({'scaler_X': scaler_X_s2, 'scaler_residual': scaler_residual,
                 'best_mae_s2': best_mae_s2, 'best_r2_s2': best_r2_s2}, f)

print(f"\n💾 Stage 2模型和scalers已保存")


# ============================================================================
# STEP 7: 最终评估 - 选择性Boosting
# ============================================================================

# Re-load Base Model for final evaluation (ensure it's on the correct device)
# Ensure CompactSensorTransformer is defined
base_model_eval = CompactSensorTransformer(
    num_inputs=num_inputs,
    num_outputs=num_outputs,
    d_model=config.STAGE1_D_MODEL,
    nhead=config.STAGE1_NHEAD,
    num_layers=config.STAGE1_NUM_LAYERS,
    dropout=config.STAGE1_DROPOUT
).to(config.DEVICE)
base_model_eval.load_state_dict(base_model_state_dict)
base_model_eval.eval()

residual_model.eval() # Residual model is already trained in this cell

print("\n" + "="*80)
print("📊 STEP 7: 最终评估（选择性Boosting）")
print("="*80)

# Prepare validation data for final evaluation (using Stage 1 scalers)
X_val_s1_scaled_eval = scaler_X_s1.transform(X_val_orig)
y_val_true_eval = y_val_orig # Use original true values for final evaluation


# Predict using Stage 1 and Stage 2 models on validation set
# Use batch processing for inference
batch_size_inference = 1024 # Adjust as needed
num_val_samples = len(X_val_s1_scaled_eval)
num_val_batches = (num_val_samples + batch_size_inference - 1) // batch_size_inference

y_val_s1_pred_list = []
residual_val_pred_list = []

with torch.no_grad():
    for i in tqdm(range(num_val_batches), desc="Final Evaluation Inference"):
        start_idx = i * batch_size_inference
        end_idx = min((i + 1) * batch_size_inference, num_val_samples)

        X_batch_s1 = torch.FloatTensor(X_val_s1_scaled_eval[start_idx:end_idx]).to(config.DEVICE)
        X_batch_s2 = torch.FloatTensor(scaler_X_s2.transform(X_val_orig[start_idx:end_idx])).to(config.DEVICE) # Scale X again for Stage 2 model

        with autocast():
            y_pred_s1_scaled = base_model_eval(X_batch_s1).cpu().numpy()
            residual_pred_scaled = residual_model(X_batch_s2).cpu().numpy()

        y_val_s1_pred_list.append(scaler_y_s1.inverse_transform(y_pred_s1_scaled))
        residual_val_pred_list.append(scaler_residual.inverse_transform(residual_pred_scaled))

y_val_s1_pred = np.vstack(y_val_s1_pred_list)
residual_val_pred = np.vstack(residual_val_pred_list)

y_val_boosted_all = y_val_s1_pred + residual_val_pred

# Calculate R² for each signal from Stage 1 on validation set
# Ensure compute_r2_safe is defined
_, signal_r2_scores_val = compute_r2_safe(y_val_true_eval, y_val_s1_pred, method='per_output_mean')


if config.USE_SELECTIVE_BOOSTING:
    weak_signals_mask_val = signal_r2_scores_val < config.R2_THRESHOLD
    num_weak_val = np.sum(weak_signals_mask_val)
    print(f"\n⭐ 选择性Boosting (Validation Set, R²阈值 = {config.R2_THRESHOLD}):")
    print(f"  需要Boosting的信号: {num_weak_val}/{len(signal_r2_scores_val)} ({num_weak_val/len(signal_r2_scores_val)*100:.1f}%)")

    y_val_selective_boosted = y_val_s1_pred.copy()
    y_val_selective_boosted[:, weak_signals_mask_val] = y_val_boosted_all[:, weak_signals_mask_val]
else:
    y_val_selective_boosted = y_val_boosted_all
    weak_signals_mask_val = np.ones(len(signal_r2_scores_val), dtype=bool)


# Final Metrics
stage1_mae_final = mean_absolute_error(y_val_true_eval, y_val_s1_pred)
stage1_r2_final, _ = compute_r2_safe(y_val_true_eval, y_val_s1_pred, method='per_output_mean')

selective_boosted_mae_final = mean_absolute_error(y_val_true_eval, y_val_selective_boosted)
selective_boosted_r2_final, _ = compute_r2_safe(y_val_true_eval, y_val_selective_boosted, method='per_output_mean')

full_boosted_mae_final = mean_absolute_error(y_val_true_eval, y_val_boosted_all)
full_boosted_r2_final, _ = compute_r2_safe(y_val_true_eval, y_val_boosted_all, method='per_output_mean')


improvement_selective_final = (stage1_mae_final - selective_boosted_mae_final) / stage1_mae_final * 100
improvement_full_final = (stage1_mae_final - full_boosted_mae_final) / stage1_mae_final * 100


print(f"\n{'='*80}")
print(f"{'指标':<25} {'Stage 1':<18} {'Full Boosting':<18} {'Selective':<18}")
print(f"{'='*80}")
print(f"{'MAE':<25} {stage1_mae_final:<18.6f} {full_boosted_mae_final:<18.6f} {selective_boosted_mae_final:<18.6f}")
print(f"{'R² Score':<25} {stage1_r2_final:<18.6f} {full_boosted_r2_final:<18.6f} {selective_boosted_r2_final:<18.6f}")
print(f"{'='*80}")
print(f"{'MAE提升':<25} {'Baseline':<18} {improvement_full_final:>+16.2f}% {improvement_selective_final:>+16.2f}%")
print(f"{'='*80}")

if improvement_selective_final > 0:
    print(f"\n✅ Boosting有效！选择性策略MAE改进 {improvement_selective_final:.2f}%")

# Plot training history (requires history_s1 from Stage 1 training)
# If history_s1 is not available, you might need to load it from training_info.pkl
# For now, assuming history_s1 is not available after kernel restart, skip plotting or load it
# plot_training_history(history_s1 if 'history_s1' in locals() else {'val_mae': [], 'val_r2': [], 'train_loss': []}, history_s2)


# ============================================================================
# STEP 8: Save Final Results and Info
# ============================================================================

print("\n" + "="*80)
print("💾 STEP 8: 保存最终结果")
print("="*80)

# Load previous training info if it exists, and update it
training_info_path = '/content/boost_training_info.pkl'
try:
    with open(training_info_path, 'rb') as f:
        training_info = pickle.load(f)
    print("✅ 已加载现有训练信息")
except FileNotFoundError:
    training_info = {}
    print("⚠️ 未找到现有训练信息，创建新信息")

# Update results
training_info['results'] = {
    'stage1_mae': stage1_mae_final,
    'stage1_r2': stage1_r2_final,
    'full_boosted_mae': full_boosted_mae_final,
    'full_boosted_r2': full_boosted_r2_final,
    'selective_boosted_mae': selective_boosted_mae_final,
    'selective_boosted_r2': selective_boosted_r2_final,
    'improvement_full': improvement_full_final,
    'improvement_selective': improvement_selective_final,
    'signal_r2_scores': signal_r2_scores_val, # Use validation set R2 for boosting decision
    'weak_signals_mask': weak_signals_mask_val,
    'num_weak_signals': np.sum(weak_signals_mask_val),
    'num_total_signals': num_outputs
}

# Update scalers
training_info['scalers'] = {
    'stage1_scaler_X': scaler_X_s1,
    'stage1_scaler_y': scaler_y_s1,
    'stage2_scaler_X': scaler_X_s2,
    'stage2_scaler_residual': scaler_residual
}

# Update history (if history_s1 was loaded or generated)
if 'history_s1' in locals():
     training_info['history'] = {'stage1': history_s1, 'stage2': history_s2}
elif 'history' in training_info and 'stage1' in training_info['history']:
     training_info['history']['stage2'] = history_s2
else:
    training_info['history'] = {'stage2': history_s2}
    print("⚠️ Stage 1 history not available for combined plot.")


# Save updated info
with open(training_info_path, 'wb') as f:
    pickle.dump(training_info, f)
print(f"✅ 完整训练信息已保存/更新: {training_info_path}")


print("\n" + "="*80)
print("✅ Stage 2训练和评估完成！")
print("="*80)

print("\n📁 生成/更新的文件:")
print("   1. /content/boost_stage2_best.pth - Stage 2最佳Checkpoint ⭐")
print("   2. /content/stage2_scalers.pkl - Stage 2 Scalers ⭐")
print("   3. /content/boost_training_info.pkl - 完整训练配置和结果 ⭐")

print("\n⭐ 性能总结 (Validation Set):")
print(f"   Stage 1 MAE: {stage1_mae_final:.6f} | R²: {stage1_r2_final:.6f}")
print(f"   Selective Boosting MAE: {selective_boosted_mae_final:.6f} | R²: {selective_boosted_r2_final:.6f}")
print(f"   MAE改进: {improvement_selective_final:+.2f}%")
print(f"   弱信号数量 (R² < {config.R2_THRESHOLD}): {np.sum(weak_signals_mask_val)}/{num_outputs}")


🚀 Stage 2 训练 (基于已保存的Stage 1)

📥 加载已保存的Base Model和Scalers
✅ Stage 1 Model State Dict加载成功
✅ Stage 1 Scalers加载成功
   最佳MAE (from scalers): inf
   最佳R² (from scalers): -inf
✅ Base Model已重建并设置为评估模式

📥 加载原始训练数据
✅ 数据加载成功 (from environment):
   输入形状: (312500, 490)
   输出形状: (312500, 164)
✅ 原始数据划分: 训练集 265,625, 验证集 46,875

📊 计算Stage 1残差

🔧 计算Stage 2残差（正确方法）

步骤1: 标准化输入

步骤2: Base模型预测（标准化空间）


Predicting Base Model Residuals:   0%|          | 0/260 [00:00<?, ?it/s]

  预测形状: (265625, 164)
  预测范围: [-32.593750, 25.265625]

步骤3: 反标准化到原始空间
  反标准化后范围: [-30.921875, 28.000000]
  真实值范围: [-122.974739, 132.202042]

步骤4: 计算残差（原始空间）
  残差形状: (265625, 164)
  残差范围: [-122.390755, 133.116104]
  残差均值: -0.004257 (应接近0)
  残差标准差: 0.744185

验证:
  残差标准差/目标标准差 = 0.7351
  ✓ Base model performed moderately.

📊 STEP 4: 准备Stage 2训练数据

🔧 计算Stage 2残差（正确方法）

步骤1: 标准化输入

步骤2: Base模型预测（标准化空间）


Predicting Base Model Residuals:   0%|          | 0/46 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [31]:
"""
================================================================================
Inference & Evaluation - 内存优化版 (Memory Optimized)
================================================================================
优化内容：
1. ✅ 批处理推理，避免一次性加载所有数据到GPU
2. ✅ 及时释放GPU内存
3. ✅ 在计算R²时也使用批处理
4. ✅ 自动根据GPU内存调整batch size

适用场景：
- GPU内存不足（CUDA OOM错误）
- 大规模测试集推理
- 多个模型同时占用GPU

默认batch_size: 512（可以根据GPU内存调整）
================================================================================
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.cuda.amp import autocast
import pickle
import warnings
import os
from datetime import datetime
import gc
warnings.filterwarnings('ignore')

# ✅ 设置环境变量避免内存碎片
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print("="*80)
print("📊 Inference & Evaluation - 内存优化版")
print("="*80)
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print("="*80)

# ============================================================================
# 配置类
# ============================================================================
class Config:
    """全局配置类"""
    TRAIN_TEST_SPLIT = 0.15
    RANDOM_SEED = 42

    # Boosting配置
    R2_THRESHOLD = 0.5
    USE_SELECTIVE_BOOSTING = True

    # ✅ 内存优化配置
    INFERENCE_BATCH_SIZE = 512  # 推理批次大小，可以根据GPU内存调整（256/512/1024）

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()

print(f"\n⚙️  配置: 设备={config.DEVICE}")
print(f"R²阈值: {config.R2_THRESHOLD}")
print(f"推理批次大小: {config.INFERENCE_BATCH_SIZE}")
print("="*80)

# ============================================================================
# GPU内存管理函数
# ============================================================================
def clear_gpu_memory():
    """清理GPU内存"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

def print_gpu_memory():
    """打印GPU内存使用情况"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"  💾 GPU内存: 已分配 {allocated:.2f} GB | 已保留 {reserved:.2f} GB")

# ============================================================================
# R²计算函数
# ============================================================================
def compute_r2_safe(y_true, y_pred, method='per_output_mean'):
    """安全的R²计算函数"""
    if y_true.ndim == 1:
        y_true = y_true.reshape(-1, 1)
        y_pred = y_pred.reshape(-1, 1)

    n_outputs = y_true.shape[1]
    per_output_r2 = np.zeros(n_outputs)

    for i in range(n_outputs):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]

        var_true = np.var(y_t)
        if var_true < 1e-10:
            per_output_r2[i] = 0.0
        else:
            try:
                per_output_r2[i] = r2_score(y_t, y_p)
            except Exception as e:
                per_output_r2[i] = -1.0

    if method == 'per_output_mean':
        valid_r2 = per_output_r2[np.isfinite(per_output_r2) & (per_output_r2 > -10)]
        r2 = np.mean(valid_r2) if len(valid_r2) > 0 else -1.0
    elif method == 'per_output_median':
        valid_r2 = per_output_r2[np.isfinite(per_output_r2) & (per_output_r2 > -10)]
        r2 = np.median(valid_r2) if len(valid_r2) > 0 else -1.0
    elif method == 'sklearn_default':
        r2 = r2_score(y_true, y_pred)
    elif method == 'global':
        r2 = r2_score(y_true.flatten(), y_pred.flatten())
    else:
        raise ValueError(f"Unknown method: {method}")

    return r2, per_output_r2


def compute_per_signal_metrics(y_true, y_pred):
    """计算每个信号的详细指标"""
    n_signals = y_true.shape[1]

    metrics = {
        'mae': np.zeros(n_signals),
        'rmse': np.zeros(n_signals),
        'r2': np.zeros(n_signals),
        'mape': np.zeros(n_signals),
        'true_mean': np.zeros(n_signals),
        'true_std': np.zeros(n_signals),
        'pred_mean': np.zeros(n_signals),
        'pred_std': np.zeros(n_signals)
    }

    for i in range(n_signals):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]

        metrics['mae'][i] = mean_absolute_error(y_t, y_p)
        metrics['rmse'][i] = np.sqrt(mean_squared_error(y_t, y_p))

        var_true = np.var(y_t)
        if var_true < 1e-10:
            metrics['r2'][i] = 0.0
        else:
            try:
                metrics['r2'][i] = r2_score(y_t, y_p)
            except:
                metrics['r2'][i] = -1.0

        non_zero_mask = np.abs(y_t) > 1e-6
        if np.sum(non_zero_mask) > 0:
            mape = np.mean(np.abs((y_t[non_zero_mask] - y_p[non_zero_mask]) / y_t[non_zero_mask])) * 100
            metrics['mape'][i] = mape
        else:
            metrics['mape'][i] = np.nan

        metrics['true_mean'][i] = np.mean(y_t)
        metrics['true_std'][i] = np.std(y_t)
        metrics['pred_mean'][i] = np.mean(y_p)
        metrics['pred_std'][i] = np.std(y_p)

    return metrics


# ============================================================================
# 模型定义
# ============================================================================
class CompactSensorTransformer(nn.Module):
    """紧凑型传感器Transformer"""
    def __init__(self, num_inputs, num_outputs, d_model=128, nhead=8,
                 num_layers=3, dropout=0.1, model_name="SST"):
        super().__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.d_model = d_model
        self.model_name = model_name

        self.input_embedding = nn.Linear(1, d_model)
        self.position_encoding = nn.Parameter(torch.randn(num_inputs, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*2,
            dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output_projection = nn.Linear(d_model, num_outputs)
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.unsqueeze(-1)
        x = self.input_embedding(x) + self.position_encoding.unsqueeze(0)
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x = self.global_pool(x).squeeze(-1)
        predictions = self.output_projection(x)
        return predictions


# ============================================================================
# 加载模型和数据
# ============================================================================
def load_models_and_scalers():
    """加载所有必需的模型和scalers"""
    print("\n" + "="*80)
    print("📂 加载模型和Scalers")
    print("="*80)

    required_files = [
        'boost_stage1_best.pth',
        'boost_stage2_best.pth',
        'stage1_scalers.pkl',
        'stage2_scalers.pkl'
    ]

    for file in required_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"❌ 未找到 {file}，请先训练模型！")

    results = None
    input_features = None
    output_features = None

    if os.path.exists('boost_results.pkl'):
        with open('boost_results.pkl', 'rb') as f:
            data = pickle.load(f)
            results = data.get('results', {})
            input_features = results.get('input_features')
            output_features = results.get('output_features')
        print("✅ 加载 boost_results.pkl")
    elif os.path.exists('boost_results_stage2_only.pkl'):
        with open('boost_results_stage2_only.pkl', 'rb') as f:
            data = pickle.load(f)
            results = data.get('results', {})
            input_features = results.get('input_features')
            output_features = results.get('output_features')
        print("✅ 加载 boost_results_stage2_only.pkl")
    else:
        print("⚠️  未找到训练结果文件，将在推理后重新计算信号R²")

    with open('stage1_scalers.pkl', 'rb') as f:
        stage1_data = pickle.load(f)

    with open('stage2_scalers.pkl', 'rb') as f:
        stage2_data = pickle.load(f)

    scalers = {
        'stage1_scaler_X': stage1_data['scaler_X'],
        'stage1_scaler_y': stage1_data['scaler_y'],
        'stage2_scaler_X': stage2_data['scaler_X'],
        'stage2_scaler_residual': stage2_data['scaler_residual']
    }

    print("✅ Scalers加载成功")

    num_inputs = scalers['stage1_scaler_X'].n_features_in_
    num_outputs = scalers['stage1_scaler_y'].n_features_in_

    print(f"\n📊 模型维度:")
    print(f"  输入特征数: {num_inputs}")
    print(f"  输出特征数: {num_outputs}")

    base_model = CompactSensorTransformer(
        num_inputs=num_inputs, num_outputs=num_outputs,
        d_model=256, nhead=16, num_layers=6, dropout=0.1,
        model_name="Stage1_Base"
    ).to(config.DEVICE)

    base_model.load_state_dict(torch.load('boost_stage1_best.pth'))
    base_model.eval()
    print(f"✅ Stage 1模型加载成功: {sum(p.numel() for p in base_model.parameters()):,} 参数")

    residual_model = CompactSensorTransformer(
        num_inputs=num_inputs, num_outputs=num_outputs,
        d_model=256, nhead=16, num_layers=6, dropout=0.15,
        model_name="Stage2_Residual"
    ).to(config.DEVICE)

    residual_model.load_state_dict(torch.load('boost_stage2_best.pth'))
    residual_model.eval()
    print(f"✅ Stage 2模型加载成功: {sum(p.numel() for p in residual_model.parameters()):,} 参数")

    print_gpu_memory()

    return base_model, residual_model, scalers, results, input_features, output_features


# ============================================================================
# ✅ 批处理推理函数 - 内存优化
# ============================================================================
def batch_inference(model, X_data, scaler_X, scaler_y, batch_size, model_name="Model"):
    """
    批处理推理函数 - 避免GPU OOM

    Args:
        model: PyTorch模型
        X_data: 输入数据 (n_samples, n_features)
        scaler_X: 输入标准化器
        scaler_y: 输出标准化器
        batch_size: 批次大小
        model_name: 模型名称（用于显示）

    Returns:
        y_pred: 预测结果（已反标准化）
    """
    model.eval()
    n_samples = X_data.shape[0]
    predictions_list = []

    print(f"\n🔮 {model_name} 批处理推理:")
    print(f"  总样本数: {n_samples:,}")
    print(f"  批次大小: {batch_size}")
    print(f"  批次数量: {(n_samples + batch_size - 1) // batch_size}")

    for i in range(0, n_samples, batch_size):
        # 获取批次数据
        end_idx = min(i + batch_size, n_samples)
        batch_X = X_data[i:end_idx]

        # 标准化
        batch_X_scaled = scaler_X.transform(batch_X)
        batch_X_tensor = torch.FloatTensor(batch_X_scaled).to(config.DEVICE)

        # 推理
        with torch.no_grad():
            with autocast():
                batch_pred_scaled = model(batch_X_tensor).cpu().numpy()

        # 反标准化
        batch_pred = scaler_y.inverse_transform(batch_pred_scaled)
        predictions_list.append(batch_pred)

        # ✅ 立即释放GPU内存
        del batch_X_tensor, batch_pred_scaled

        # 每10个batch清理一次
        if (i // batch_size + 1) % 10 == 0:
            clear_gpu_memory()

        # 显示进度
        if (i // batch_size + 1) % 20 == 0 or end_idx == n_samples:
            print(f"  进度: {end_idx:,}/{n_samples:,} ({end_idx/n_samples*100:.1f}%)")

    # 合并所有批次的预测
    y_pred = np.vstack(predictions_list)

    # ✅ 最终清理
    clear_gpu_memory()

    print(f"  ✅ 完成！输出形状: {y_pred.shape}")
    print_gpu_memory()

    return y_pred


# ============================================================================
# 推理函数 - 使用批处理
# ============================================================================
def inference_with_boosting(X_test, base_model, residual_model, scalers,
                           signal_r2_scores=None, r2_threshold=None):
    """
    使用Boosting进行推理 - 内存优化版

    Args:
        X_test: 测试数据 (n_samples, n_features)
        base_model: Stage 1模型
        residual_model: Stage 2模型
        scalers: 标准化器字典
        signal_r2_scores: 各信号的R²分数（用于选择性Boosting）
        r2_threshold: R²阈值

    Returns:
        y_pred_stage1: Stage 1的预测结果
        y_pred_boosted: Boosting后的预测结果
        boosting_mask: 哪些信号被Boosting了
    """
    if r2_threshold is None:
        r2_threshold = config.R2_THRESHOLD

    print("\n" + "="*80)
    print("🔮 执行推理 - 内存优化版")
    print("="*80)

    # ✅ Stage 1批处理推理
    y_pred_stage1 = batch_inference(
        base_model, X_test,
        scalers['stage1_scaler_X'],
        scalers['stage1_scaler_y'],
        config.INFERENCE_BATCH_SIZE,
        "Stage 1"
    )

    # ✅ Stage 2批处理推理
    residual_pred = batch_inference(
        residual_model, X_test,
        scalers['stage2_scaler_X'],
        scalers['stage2_scaler_residual'],
        config.INFERENCE_BATCH_SIZE,
        "Stage 2"
    )

    # 全量Boosting
    y_pred_full_boosted = y_pred_stage1 + residual_pred

    # 选择性Boosting
    if config.USE_SELECTIVE_BOOSTING and signal_r2_scores is not None:
        boosting_mask = signal_r2_scores < r2_threshold
        num_boosted = np.sum(boosting_mask)
        num_total = len(boosting_mask)

        print(f"\n⭐ 选择性Boosting:")
        print(f"  R²阈值: {r2_threshold}")
        print(f"  被Boosting的信号: {num_boosted}/{num_total} ({num_boosted/num_total*100:.1f}%)")
        print(f"  保持Stage 1的信号: {num_total-num_boosted}/{num_total} ({(num_total-num_boosted)/num_total*100:.1f}%)")

        y_pred_boosted = y_pred_stage1.copy()
        y_pred_boosted[:, boosting_mask] = y_pred_full_boosted[:, boosting_mask]
    else:
        print(f"\n⚠️  未使用选择性Boosting，所有信号都应用残差")
        y_pred_boosted = y_pred_full_boosted
        boosting_mask = np.ones(y_pred_stage1.shape[1], dtype=bool)

    return y_pred_stage1, y_pred_boosted, boosting_mask


# ============================================================================
# 生成评估报告
# ============================================================================
def generate_evaluation_report(y_test, y_pred_stage1, y_pred_boosted,
                              boosting_mask, output_features=None):
    """生成详细的评估报告CSV"""
    print("\n" + "="*80)
    print("📊 生成评估报告")
    print("="*80)

    n_signals = y_test.shape[1]

    if output_features is None:
        output_features = [f"Signal_{i}" for i in range(n_signals)]

    # 计算Stage 1指标
    print("\n计算Stage 1指标...")
    metrics_stage1 = compute_per_signal_metrics(y_test, y_pred_stage1)

    # 计算Boosting后指标
    print("计算Boosting后指标...")
    metrics_boosted = compute_per_signal_metrics(y_test, y_pred_boosted)

    # 创建详细报告DataFrame
    report_data = []

    for i in range(n_signals):
        row = {
            'Signal_Index': i,
            'Signal_Name': output_features[i] if i < len(output_features) else f"Signal_{i}",

            # Stage 1指标
            'Stage1_R2': metrics_stage1['r2'][i],
            'Stage1_MAE': metrics_stage1['mae'][i],
            'Stage1_RMSE': metrics_stage1['rmse'][i],
            'Stage1_MAPE': metrics_stage1['mape'][i],

            # Boosting后指标
            'Boosted_R2': metrics_boosted['r2'][i],
            'Boosted_MAE': metrics_boosted['mae'][i],
            'Boosted_RMSE': metrics_boosted['rmse'][i],
            'Boosted_MAPE': metrics_boosted['mape'][i],

            # 改进量
            'R2_Improvement': metrics_boosted['r2'][i] - metrics_stage1['r2'][i],
            'MAE_Improvement': metrics_stage1['mae'][i] - metrics_boosted['mae'][i],
            'RMSE_Improvement': metrics_stage1['rmse'][i] - metrics_boosted['rmse'][i],

            # 改进百分比
            'MAE_Improvement_Pct': ((metrics_stage1['mae'][i] - metrics_boosted['mae'][i]) /
                                   (metrics_stage1['mae'][i] + 1e-10) * 100),
            'RMSE_Improvement_Pct': ((metrics_stage1['rmse'][i] - metrics_boosted['rmse'][i]) /
                                    (metrics_stage1['rmse'][i] + 1e-10) * 100),

            # 真实值统计
            'True_Mean': metrics_stage1['true_mean'][i],
            'True_Std': metrics_stage1['true_std'][i],

            # 是否被Boosting
            'Is_Boosted': boosting_mask[i],

            # 信号质量分类
            'Stage1_Quality': 'Excellent' if metrics_stage1['r2'][i] >= 0.9
                            else 'Good' if metrics_stage1['r2'][i] >= 0.7
                            else 'Fair' if metrics_stage1['r2'][i] >= 0.5
                            else 'Poor',
            'Boosted_Quality': 'Excellent' if metrics_boosted['r2'][i] >= 0.9
                             else 'Good' if metrics_boosted['r2'][i] >= 0.7
                             else 'Fair' if metrics_boosted['r2'][i] >= 0.5
                             else 'Poor'
        }

        report_data.append(row)

    df_report = pd.DataFrame(report_data)

    # 创建整体对比摘要
    overall_stage1_r2, _ = compute_r2_safe(y_test, y_pred_stage1)
    overall_boosted_r2, _ = compute_r2_safe(y_test, y_pred_boosted)

    overall_stage1_mae = mean_absolute_error(y_test, y_pred_stage1)
    overall_boosted_mae = mean_absolute_error(y_test, y_pred_boosted)

    overall_stage1_rmse = np.sqrt(mean_squared_error(y_test, y_pred_stage1))
    overall_boosted_rmse = np.sqrt(mean_squared_error(y_test, y_pred_boosted))

    summary_data = {
        'Metric': ['R²', 'MAE', 'RMSE', 'Num_Signals_Boosted'],
        'Stage1': [overall_stage1_r2, overall_stage1_mae, overall_stage1_rmse, 0],
        'Boosted': [overall_boosted_r2, overall_boosted_mae, overall_boosted_rmse,
                   np.sum(boosting_mask)],
        'Improvement': [
            overall_boosted_r2 - overall_stage1_r2,
            overall_stage1_mae - overall_boosted_mae,
            overall_stage1_rmse - overall_boosted_rmse,
            np.sum(boosting_mask)
        ],
        'Improvement_Pct': [
            (overall_boosted_r2 - overall_stage1_r2) / (abs(overall_stage1_r2) + 1e-10) * 100,
            (overall_stage1_mae - overall_boosted_mae) / (overall_stage1_mae + 1e-10) * 100,
            (overall_stage1_rmse - overall_boosted_rmse) / (overall_stage1_rmse + 1e-10) * 100,
            np.sum(boosting_mask) / len(boosting_mask) * 100
        ]
    }

    df_summary = pd.DataFrame(summary_data)

    # 保存CSV
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    report_filename = f"signal_evaluation_report_{timestamp}.csv"
    df_report.to_csv(report_filename, index=False)
    print(f"\n✅ 详细评估报告已保存: {report_filename}")

    summary_filename = f"boost_comparison_summary_{timestamp}.csv"
    df_summary.to_csv(summary_filename, index=False)
    print(f"✅ 整体对比摘要已保存: {summary_filename}")

    # 打印摘要
    print("\n" + "="*80)
    print("📊 评估摘要")
    print("="*80)
    print(f"\n总信号数: {n_signals}")
    print(f"被Boosting的信号数: {np.sum(boosting_mask)} ({np.sum(boosting_mask)/n_signals*100:.1f}%)")
    print(f"\n{'指标':<15} {'Stage 1':<15} {'Boosted':<15} {'改进':<15}")
    print("-" * 60)
    print(f"{'R²':<15} {overall_stage1_r2:<15.6f} {overall_boosted_r2:<15.6f} "
          f"{overall_boosted_r2 - overall_stage1_r2:+<15.6f}")
    print(f"{'MAE':<15} {overall_stage1_mae:<15.6f} {overall_boosted_mae:<15.6f} "
          f"{overall_stage1_mae - overall_boosted_mae:+<15.6f}")
    print(f"{'RMSE':<15} {overall_stage1_rmse:<15.6f} {overall_boosted_rmse:<15.6f} "
          f"{overall_stage1_rmse - overall_boosted_rmse:+<15.6f}")
    print("="*80)

    # Top改进信号
    print("\n📈 Top 10 改进最大的信号 (按R²改进排序):")
    top_improved = df_report.nlargest(10, 'R2_Improvement')[
        ['Signal_Name', 'Stage1_R2', 'Boosted_R2', 'R2_Improvement', 'Is_Boosted']
    ]
    print(top_improved.to_string(index=False))

    return df_report, df_summary


# ============================================================================
# 主推理和评估流程
# ============================================================================
def run_inference_and_evaluation(X, y, input_features=None, output_features=None):
    """运行完整的推理和评估流程 - 内存优化版"""
    print("\n" + "="*80)
    print("🚀 开始推理和评估流程 - 内存优化版")
    print("="*80)

    # 1. 加载模型
    base_model, residual_model, scalers, train_results, \
        loaded_input_features, loaded_output_features = load_models_and_scalers()

    if output_features is None and loaded_output_features is not None:
        output_features = loaded_output_features
    if input_features is None and loaded_input_features is not None:
        input_features = loaded_input_features

    # 2. 划分测试集
    print("\n" + "="*80)
    print("📊 准备测试数据")
    print("="*80)

    _, X_test, _, y_test = train_test_split(
        X, y, test_size=config.TRAIN_TEST_SPLIT, random_state=config.RANDOM_SEED
    )

    print(f"测试集大小: {len(X_test):,} 样本")
    print(f"输入特征数: {X_test.shape[1]}")
    print(f"输出特征数: {y_test.shape[1]}")

    # 3. 获取或计算信号R²分数
    if train_results is not None and 'signal_r2_scores' in train_results:
        signal_r2_scores = train_results['signal_r2_scores']
        print(f"\n✅ 使用训练时的信号R²分数")
    else:
        print(f"\n⚠️  未找到训练时的信号R²分数，将基于测试集重新计算...")
        print(f"   使用批处理避免GPU OOM...")

        # ✅ 使用批处理计算R²分数
        y_pred_stage1_temp = batch_inference(
            base_model, X_test,
            scalers['stage1_scaler_X'],
            scalers['stage1_scaler_y'],
            config.INFERENCE_BATCH_SIZE,
            "Stage 1 (R²计算)"
        )

        _, signal_r2_scores = compute_r2_safe(y_test, y_pred_stage1_temp)
        print(f"✅ 信号R²分数计算完成")

        # ✅ 清理临时数据
        del y_pred_stage1_temp
        clear_gpu_memory()

    # 4. 推理
    y_pred_stage1, y_pred_boosted, boosting_mask = inference_with_boosting(
        X_test, base_model, residual_model, scalers,
        signal_r2_scores, config.R2_THRESHOLD
    )

    # 5. 生成评估报告
    df_report, df_summary = generate_evaluation_report(
        y_test, y_pred_stage1, y_pred_boosted,
        boosting_mask, output_features
    )

    # 6. 返回结果
    results = {
        'y_test': y_test,
        'y_pred_stage1': y_pred_stage1,
        'y_pred_boosted': y_pred_boosted,
        'boosting_mask': boosting_mask,
        'signal_r2_scores': signal_r2_scores,
        'df_report': df_report,
        'df_summary': df_summary
    }

    print("\n" + "="*80)
    print("✅ 推理和评估流程完成！")
    print("="*80)
    print_gpu_memory()

    return results


# ============================================================================
# 使用说明
# ============================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("📋 使用说明 - 内存优化版")
    print("="*80)
    print("""
    本脚本是内存优化版本，解决GPU OOM问题。

    优化特性：
    1. ✅ 批处理推理（默认batch_size=512）
    2. ✅ 及时释放GPU内存
    3. ✅ 自动内存碎片整理
    4. ✅ 进度显示

    使用方法：

    from inference_and_evaluation_memory_optimized import run_inference_and_evaluation

    # 如果仍然OOM，可以调整batch_size
    # 在导入后修改配置：
    import inference_and_evaluation_memory_optimized as inf
    inf.config.INFERENCE_BATCH_SIZE = 256  # 减小batch size

    results = run_inference_and_evaluation(X, y, input_features, output_features)

    调整建议：
    - GPU 40GB: batch_size=1024
    - GPU 24GB: batch_size=512 (默认)
    - GPU 16GB: batch_size=256
    - GPU 12GB: batch_size=128
    """)
    print("="*80)

📊 Inference & Evaluation - 内存优化版
PyTorch版本: 2.8.0+cu126
CUDA可用: True
GPU: NVIDIA A100-SXM4-40GB
GPU内存: 39.56 GB

⚙️  配置: 设备=cuda
R²阈值: 0.5
推理批次大小: 512

📋 使用说明 - 内存优化版

    本脚本是内存优化版本，解决GPU OOM问题。

    优化特性：
    1. ✅ 批处理推理（默认batch_size=512）
    2. ✅ 及时释放GPU内存
    3. ✅ 自动内存碎片整理
    4. ✅ 进度显示

    使用方法：

    from inference_and_evaluation_memory_optimized import run_inference_and_evaluation

    # 如果仍然OOM，可以调整batch_size
    # 在导入后修改配置：
    import inference_and_evaluation_memory_optimized as inf
    inf.config.INFERENCE_BATCH_SIZE = 256  # 减小batch size

    results = run_inference_and_evaluation(X, y, input_features, output_features)

    调整建议：
    - GPU 40GB: batch_size=1024
    - GPU 24GB: batch_size=512 (默认)
    - GPU 16GB: batch_size=256
    - GPU 12GB: batch_size=128
    


In [32]:
X = df_train_clean[INPUT_FEATURES_CLEAN].values
y = df_train_clean[OUTPUT_FEATURES_CLEAN].values
input_features = INPUT_FEATURES_CLEAN
output_features = OUTPUT_FEATURES_CLEAN
results = run_inference_and_evaluation(X, y, input_features, output_features)


🚀 开始推理和评估流程 - 内存优化版

📂 加载模型和Scalers
⚠️  未找到训练结果文件，将在推理后重新计算信号R²
✅ Scalers加载成功

📊 模型维度:
  输入特征数: 490
  输出特征数: 164
✅ Stage 1模型加载成功: 3,330,724 参数
✅ Stage 2模型加载成功: 3,330,724 参数
  💾 GPU内存: 已分配 0.15 GB | 已保留 0.21 GB

📊 准备测试数据
测试集大小: 93,750 样本
输入特征数: 490
输出特征数: 164

⚠️  未找到训练时的信号R²分数，将基于测试集重新计算...
   使用批处理避免GPU OOM...

🔮 Stage 1 (R²计算) 批处理推理:
  总样本数: 93,750
  批次大小: 512
  批次数量: 184
  进度: 10,240/93,750 (10.9%)
  进度: 20,480/93,750 (21.8%)
  进度: 30,720/93,750 (32.8%)
  进度: 40,960/93,750 (43.7%)
  进度: 51,200/93,750 (54.6%)
  进度: 61,440/93,750 (65.5%)
  进度: 71,680/93,750 (76.5%)
  进度: 81,920/93,750 (87.4%)
  进度: 92,160/93,750 (98.3%)
  进度: 93,750/93,750 (100.0%)
  ✅ 完成！输出形状: (93750, 164)
  💾 GPU内存: 已分配 0.15 GB | 已保留 0.19 GB
✅ 信号R²分数计算完成

🔮 执行推理 - 内存优化版

🔮 Stage 1 批处理推理:
  总样本数: 93,750
  批次大小: 512
  批次数量: 184
  进度: 10,240/93,750 (10.9%)
  进度: 20,480/93,750 (21.8%)
  进度: 30,720/93,750 (32.8%)
  进度: 40,960/93,750 (43.7%)
  进度: 51,200/93,750 (54.6%)
  进度: 61,440/93,750 (65.5%)
  进度: 71,680/93,750 (76.